## Steps

### Part 1: Prepare a data set for hotels with data from Google Local, Reviews and Airport codes 
This isn't straightforward -- the many steps are detailed below.
1. Read in reviews data set, get distinct hotels -- this is considered a new data set in the context of Part 1.
2. Create a Normalised_Hotel_Name column, we use this column to join this data set with the Google Local data set. This column has trailing and leading spaces removed, punctuations cut out, and the text converted to lower case.
3. Read in the already cleaned and reduced (only Europe) Google Local businesses data set. It is now in a valid JSONLines format, unlike in the original file.
4. Reorder the columns.
5. Like in Step 2, normalise the hotel name for this data set.
6. **Fuzzy join**: Fuzzy Join on the normalised hotel name column of the 2 data sets -- google local and distinct hotels in the reviews data set. The fuzzy join is done by considering a Levenshtein distance < 3.  This will produce many false positives.
7. The plan is to remove false positives by getting the distance between the respective latitude and longitude columns  in the 2 data sets. This is done using the geodesic distance. But before that, we have to fix some issues with these columns. 
8. **Fix latitude and longitude data issue 1: NA values from reviews data set**. We find that there are 16 rows like this, but 10 are for a single hotel, 'Hotel Atlanta'. We drop these rows as they are false positives. For the other 6 rows, we get the values from the lat and lng columns of the Google Local data set and insert it in the latitude and longitude columns of the distinct hotels data set.
9. **Fix latitude and longitude data issue 2: latitudes and longitudes outside the [-90,90] range**. These values are from the lat and lng columns in the Google Local data set. It seems like all these values are missing a decimal -- they seem to be multiplied by 1,000,000. 
10. Now, we are ready to correct the false positives prolem from step 6. We create a new column 'distance' using the latitude and longitude columns (from the reviews data set), and the lat and lng columns (from the google local data set). We do this using the **geodesic distance** using the geopy library.
11. We filter hotels with a distance < 0.3 (km).
12. As we don't have all hotels from the reviews data set in the google local data set, we need the remaining hotels as well from the reviews data set. To get these hotels, we perform a left join using the Hotel_Name, lat and lng columns in the data sets (data set in Step 1 and data set in Step 11), and drop the columns which are not needed. These additional rows need to be added to the data frame.
13. **Data issue: the hotel name and address in the reviews data set does not render latin characters properly**. Use the hotel name from the google local data set, where available to populate a new hotel name column. When it is not available, use the hotel name from the reviews data set.
14. Get the country from the hotel address string column.
15. Remove all adiditional columns which won't be written to the staging table: latitude, longitude, distance the second hotel name, second hotel address. 
16. Rename the columns as required.
17. **Assign a new primary key column to the hotels: HotelID**
18. Data Fix: Manually fill in the latitude and longitude of 16 hotels for which they are both 'NA'. I get these values from Google (manually) as there are only 16 rows.

### Part 2: Get the nearest airport to each hotel from Airport codes, get the final version of the Hotels and Airports data frames
19. Read in airport_codes and filter so that we have airports in the EU. Also, **Data issue: remove spam records**
20. Add in the country based on the countryiso.csv ISO-Country name mapping file.
21. Now, we need to join the airport codes data with the df_google_hotels data based on Country. We only want to insert the 'AirportID', 'latitude' and 'longitude' columns into the df_google_hotels data frame along with all the existing columns. Obviously, we will get a lot of duplicate data as every row in a particular country will join with every airport in that country.
22. Calculate the geodesic distance using exactly the same method as before
23. Use a Window function by partitioning over the HotelID column. This will be used to get the minimum distance over each window. The airport with the minimum distance from the hotel is the nearest airport. This is a two-step process. First, we create a new column min_distance which contains the minimum distance to an airport for a window. Then, we filter rows where min_distance=distance to get the final results.
24. Create a lookup data frame with columns hotel_id, Hotel_name (from original data frame), lat and lng. This will be used to assign Hotel IDs as FKs to the original reviews data set in Part 3.
25. Create final version of the clean df_google_hotels_with_airport_id data frame. In this step, we drop unnecessary columns, rename and reorder columns. This is saved in JSONL format, will be loaded into S3 in Part 4, and will be loaded into a Redshift staging table from there.
26. Create final version of the df_airport_codes data frame. In this step, we drop unnecessary columns, rename and reorder columns This is saved in JSONL format, will be loaded into S3 in Part 4, and will be directly loaded into a Redshift dimension table from there. 

26b. **Data issue** : Some rows contain a string which starts with 'Duplicate' for the hotel name. These are duplicate rows and can be removed.

### Part 3: Assing Hotel IDs from the created data frames to the Reviews data set (foreign keys), create the final version of the reviews data frame. 
27. Our first goal is to join the df_hotelid_lookup data frame with the reviews data frame on Hotel_Name, lat and lng. However, we need to fix the same issues we fixed while building the data frame for the join to take place successfully. In other words, we need to do something about the rows which have 'NA' lat/long values. Perform the same process as Step 18. Drop rows with Hotel_Name='Hotel Atlanta'. Add in the latitude and longitude for the rows with NA, i.e., for the same 16 hotels that were manipulated in Step 18. 
28. Join the df_hotelid_lookup data frame with the reviews data frame on (Hotel_Name, lat, lng).
29. As we have the HotelId now in our data frame, we can drop all the other information like Hotel Name, lat, lng and Hotel Address. These are already identified by the Hotel ID. Also, the days since review is a useless field and can be removed.
30. As we saw in the Explore_reviews notebook, the string columns have leading and spaces. These need to be stripped. Also, some string columns have empty spaces as their values. These values need to be replaced with None -- we want a Null to be added into the final database.
31. Change the datatype of the Review_Date field from string to date.
32. Create the final version of the df_hotel_reviews data frame. We  rename and reorder columns. This will be loaded into S3 in Part 4, and will be loaded into a Redshift staging table from there.
33. Write the reviews dataframe to disk in JSONL format.
34. Read in the created CountryIndicators.csv File (created in the **CombineAdditionalCountryData notebook**), convert it into JSONL format, and write it into a local directory.
### Part 4: Write parquet data sets into a new s3 bucket using the AWS CLI 
35. All 4 data sets need to be written into S3 after creating a bucket. I convert all the data sets into JSONL formatand also create JSONPath files to describe their schemas (this is required for Redshift, as 'auto' json doesn't work well. These tasks are done using the AWS CLI. The commands are given in Part 4.

In [2]:
! pip install geopy
! pip install https://github.com/elyase/geotext/archive/master.zip

    100% |████████████████████████████████| 122kB 4.4MB/s ta 0:00:01
     \ 4.3MB 56.7MB/s    \ 225kB 3.8MB/s     | 440kB 3.8MB/s
  Running setup.py bdist_wheel for geotext ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-uo0y5760/wheels/f5/e3/84/31638877059a434d8601a764fc7565f2a9f7b6fb327085191e
Successfully built geotext


In [3]:
import collections
import json
import ast
from tqdm import tqdm
from geotext import GeoText
from geopy import distance
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, DoubleType, DecimalType
from gensim.parsing import preprocessing
import boto3
import configparser

Read in the Reviews data set. In this notebook, we are going to get 3 fields: the hotel name, latitude and longitude, and normalise the hotel name. 

In [4]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

## Part 1: Prepare a data set for hotels with data from Google Local, Reviews and Airport codes 

Step 1. Read in reviews data set, get distinct hotels -- this is considered a new data set in the context of Part 1.

In [91]:
# Read in the data here
df_hotel_reviews = spark.read.csv('Data/Original/Hotel_Reviews.csv', header=True, inferSchema=True)
df_hotel_reviews.printSchema()


root
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: string (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: string (nullable = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)



In [85]:
dist_hotel_columns = ['Hotel_Name', 'Hotel_Address', 'lat', 'lng']
df_distinct_hotels = df_hotel_reviews.dropDuplicates(subset=dist_hotel_columns)

In [86]:
df_distinct_hotels.head()

Row(Hotel_Address='Savoyenstra e 2 16 Ottakring 1160 Vienna Austria', Additional_Number_of_Scoring=86, Review_Date='7/31/2017', Average_Score=8.3, Hotel_Name='Austria Trend Hotel Schloss Wilhelminenberg Wien', Reviewer_Nationality=' Netherlands ', Negative_Review=' Although the building looks majestic on the outside the inner hallways on the higher floors could use some renovation old carpets walls and ceilings need a freshen up The wooden floors expecially in the hallways are quite squeaky Our room was in decent shape but had no airconditioning a fan was provided though We stayed during a 30 degree celcius period so the room was rather warm and hard to ventilate because of the small single window ', Review_Total_Negative_Word_Counts=79, Total_Number_of_Reviews=1558, Positive_Review=' Excellent location with spectacular view over the city provided your room is on the backside of the building Local bus to the city stops right in front of the hotel Ample free outdoor parking on the hotel

In [87]:
# Remove all columns except the ones we need
df_distinct_hotels = df_distinct_hotels.select(dist_hotel_columns)
print((df_distinct_hotels.count(), len(df_distinct_hotels.columns)))

(1494, 4)


In [88]:
df_distinct_hotels.limit(5).toPandas()

,Hotel_Name,Hotel_Address,lat,lng
0,Hilton London Paddington,146 Praed Street Westminster Borough London W2...,51.515632,-0.175737
1,Millennium Hotel London Knightsbridge,17 25 Sloane Street Knightsbridge Kensington a...,51.4999683,-0.1599927
2,Grand Pigalle Hotel,29 rue Victor Mass 9th arr 75009 Paris France,48.880856,2.3375085
3,St George Hotel,49 Gloucester Place Marble Arch Westminster Bo...,51.5182768,-0.1583511
4,H tel Regent s Garden,6 Rue Pierre Demours 17th arr 75017 Paris France,48.8797513,2.2919634


Step 2. Create a Normalised_Hotel_Name column, we use this column to join this data set with the Google Local data set. This column has trailing and leading spaces removed, punctuations cut out, and the text converted to lower case.

In [89]:
@F.udf(returnType=StringType())
def normalise_string(text):
    """ Normalises a string by first replacing punctuations with a space,
    and then stripping leading and trailing spaces, and lower casing the text"""
    return preprocessing.strip_punctuation(text).strip().lower()

In [90]:
df_distinct_hotels = df_distinct_hotels.withColumn('Normalised_Hotel_Name', normalise_string(F.col('Hotel_Name')))

In [91]:
df_distinct_hotels.limit(5).toPandas()

,Hotel_Name,Hotel_Address,lat,lng,Normalised_Hotel_Name
0,Hilton London Paddington,146 Praed Street Westminster Borough London W2...,51.515632,-0.175737,hilton london paddington
1,Millennium Hotel London Knightsbridge,17 25 Sloane Street Knightsbridge Kensington a...,51.4999683,-0.1599927,millennium hotel london knightsbridge
2,Grand Pigalle Hotel,29 rue Victor Mass 9th arr 75009 Paris France,48.880856,2.3375085,grand pigalle hotel
3,St George Hotel,49 Gloucester Place Marble Arch Westminster Bo...,51.5182768,-0.1583511,st george hotel
4,H tel Regent s Garden,6 Rue Pierre Demours 17th arr 75017 Paris France,48.8797513,2.2919634,h tel regent s garden


Step 3. Read in the already cleaned and reduced Google Local businesses data set. It is now in a valid JSONLines format, unlike in the original file.

In [12]:
google_businesses_df = spark.read.json('Data/Cleaned/google_places_cleaned.jsonl')

In [13]:
col_order = ['name', 'address', 'city', 'country', 'gPlusPlaceId', 'phone',
             'price', 'latitude', 'longitude', 'MondayHours', 'TuesdayHours',
            'WednesdayHours', 'ThursdayHours', 'FridayHours', 'SaturdayHours',
            'SundayHours']
print(len(col_order), len(google_businesses_df.columns))

16 16


Step 4. Reorder the columns

In [14]:
google_businesses_df = google_businesses_df.select(col_order)
google_businesses_df.limit(6).toPandas()

,name,address,city,country,gPlusPlaceId,phone,price,latitude,longitude,MondayHours,TuesdayHours,WednesdayHours,ThursdayHours,FridayHours,SaturdayHours,SundayHours
0,Café Langereis,"[Amstel 202, 1017 AH Amsterdam, Netherlands]",None,Netherlands,110421660062903864339,020 785 0641,None,52.366119,4.900194,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--3:00 am,11:00 am--3:00 am,11:00 am--1:00 am
1,LA ROMA,"[Calle Zaragoza, 16, 11402 Jerez de la Fronter...",None,Spain,103062340852821108165,856 92 73 32,None,36.688028,-6.134660,1:00--4:00 pm,1:00--4:00 pm,1:00--4:00 pm,1:00--4:00 pm,1:00--4:00 pm,1:00--4:00 pm,1:00--4:00 pm
2,HSBC Head Quarters,"[8 Canada Sq, London E14 5HQ, United Kingdom]",London,United Kingdom,102297455696045037925,020 7991 8888,None,51.506582,-0.016885,8:00 am--7:00 pm,9:30 am--7:00 pm,8:00 am--7:00 pm,8:00 am--7:00 pm,9:00 am--7:00 pm,10:00 am--6:00 pm,12:00--6:00 pm
3,Scuola Media Statale Antonino De Stefano,"[Via Salvatore Caruso, 1, 91016 Casa Santa, Er...",Casa Santa,Italy,112728625344817603614,0923 561446,None,38.022947,12.547262,None,None,None,None,None,None,None
4,Medical Óptica La Paz,"[La Paz Kalea, 27, 01004 Vitoria-Gasteiz, Arab...",La Paz,Spain,116613597224081757967,945 56 79 18,None,42.846554,-2.667956,None,None,None,None,None,None,None
5,Bar Sport Internet Point,"[Via Dante, 14 39042 Bressanone BZ, Italy]",None,Italy,100602226910017835754,0472 831537,None,46.717622,11.651580,7:00 am--10:00 pm,7:00 am--11:30 pm,7:00 am--11:30 pm,7:00 am--11:30 pm,7:00 am--11:30 pm,5:30--11:30 pm,2:30--11:30 pm


Step 5. Like in Step 2, normalise the hotel name for this data set.

In [15]:
google_businesses_df = google_businesses_df.withColumn('NormalisedHotelName', normalise_string(F.col('name')))

In [16]:
google_businesses_df.limit(6).toPandas()

,name,address,city,country,gPlusPlaceId,phone,price,latitude,longitude,MondayHours,TuesdayHours,WednesdayHours,ThursdayHours,FridayHours,SaturdayHours,SundayHours,NormalisedHotelName
0,Café Langereis,"[Amstel 202, 1017 AH Amsterdam, Netherlands]",None,Netherlands,110421660062903864339,020 785 0641,None,52.366119,4.900194,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--1:00 am,11:00 am--3:00 am,11:00 am--3:00 am,11:00 am--1:00 am,café langereis
1,LA ROMA,"[Calle Zaragoza, 16, 11402 Jerez de la Fronter...",None,Spain,103062340852821108165,856 92 73 32,None,36.688028,-6.134660,1:00--4:00 pm,1:00--4:00 pm,1:00--4:00 pm,1:00--4:00 pm,1:00--4:00 pm,1:00--4:00 pm,1:00--4:00 pm,la roma
2,HSBC Head Quarters,"[8 Canada Sq, London E14 5HQ, United Kingdom]",London,United Kingdom,102297455696045037925,020 7991 8888,None,51.506582,-0.016885,8:00 am--7:00 pm,9:30 am--7:00 pm,8:00 am--7:00 pm,8:00 am--7:00 pm,9:00 am--7:00 pm,10:00 am--6:00 pm,12:00--6:00 pm,hsbc head quarters
3,Scuola Media Statale Antonino De Stefano,"[Via Salvatore Caruso, 1, 91016 Casa Santa, Er...",Casa Santa,Italy,112728625344817603614,0923 561446,None,38.022947,12.547262,None,None,None,None,None,None,None,scuola media statale antonino de stefano
4,Medical Óptica La Paz,"[La Paz Kalea, 27, 01004 Vitoria-Gasteiz, Arab...",La Paz,Spain,116613597224081757967,945 56 79 18,None,42.846554,-2.667956,None,None,None,None,None,None,None,medical óptica la paz
5,Bar Sport Internet Point,"[Via Dante, 14 39042 Bressanone BZ, Italy]",None,Italy,100602226910017835754,0472 831537,None,46.717622,11.651580,7:00 am--10:00 pm,7:00 am--11:30 pm,7:00 am--11:30 pm,7:00 am--11:30 pm,7:00 am--11:30 pm,5:30--11:30 pm,2:30--11:30 pm,bar sport internet point


In [17]:
print("No. of rows currently in Google Local Businesses dataset = ", google_businesses_df.count())
print("No. of unique hotels in hotel reviews data set = ", df_distinct_hotels.count())

No. of rows currently in Google Local Businesses dataset =  464906
No. of unique hotels in hotel reviews data set =  1494


Test with normal (not fuzzy join).

I use the google_businesses_df as the left table, df_distinct_hotels as the right table, and try to inner join the two tables (dfs) on the hotel name. 

In [31]:
df_google_hotels = google_businesses_df.join(df_distinct_hotels,
                                             google_businesses_df.name == df_distinct_hotels.Hotel_Name)

In [32]:
df_google_hotels.count()

581

In [33]:
df_google_hotels_norm = google_businesses_df.join(df_distinct_hotels,
                                             google_businesses_df.NormalisedHotelName == df_distinct_hotels.Normalised_Hotel_Name)

In [34]:
df_google_hotels_norm.count()

611

In [36]:
df_google_hotels_norm.limit(10).toPandas()

,name,address,city,country,gPlusPlaceId,phone,price,latitude,longitude,MondayHours,...,ThursdayHours,FridayHours,SaturdayHours,SundayHours,NormalisedHotelName,Hotel_Name,Hotel_Address,lat,lng,Normalised_Hotel_Name
0,NH Carlton Amsterdam,"[Vijzelstraat 4, 1017 HK Amsterdam, Netherlands]",None,Netherlands,115445721912297534396,020 622 2266,None,52.366360,4.892972,None,...,None,None,None,None,nh carlton amsterdam,NH Carlton Amsterdam,Vijzelstraat 4 Amsterdam City Center 1017 HK A...,52.3663727,4.8930505,nh carlton amsterdam
1,Le Meridien Piccadilly,"[21 Piccadilly, London W1J 0BH, United Kingdom]",London,United Kingdom,103834982949277428473,020 7734 8000,None,51.509438,-0.136648,None,...,None,None,None,None,le meridien piccadilly,Le Meridien Piccadilly,21 Piccadilly Westminster Borough London W1J 0...,51.5094835,-0.1362538,le meridien piccadilly
2,Starhotels Anderson,"[Piazza Luigi di Savoia, 20, 20124 Milano, Italy]",Milano,Italy,101216097738766780510,02 669 0141,None,45.485177,9.206873,None,...,None,None,None,None,starhotels anderson,Starhotels Anderson,Piazza Luigi di Savoia 20 Central Station 2012...,45.4852549,9.2069501,starhotels anderson
3,Room Mate Aitana,"[IJdok, 1013 MM Amsterdam, Netherlands]",None,Netherlands,107517641782692436781,020 891 4800,None,52.384276,4.894653,None,...,None,None,None,None,room mate aitana,Room Mate Aitana,IJdock 6 Amsterdam City Center 1013 MM Amsterd...,52.3846059,4.8941866,room mate aitana
4,Catalonia Barcelona Plaza,"[Plaza España, 6-8, 08014 Barcelona, Spain]",Barcelona,Spain,104385384533610737798,934 26 26 00,None,41.375339,2.148359,None,...,None,None,None,None,catalonia barcelona plaza,Catalonia Barcelona Plaza,Plaza Espa a 6 8 Sants Montju c 08014 Barcelon...,41.3750293,2.1491172,catalonia barcelona plaza
5,La Villa Maillot,"[143 Avenue de Malakoff, 75116 Paris, France]",Paris,France,113033302845655399954,01 53 64 52 52,None,48.875582,2.283694,None,...,None,None,None,None,la villa maillot,La Villa Maillot,143 Avenue De Malakoff 16th arr 75016 Paris Fr...,48.875551,2.2835191,la villa maillot
6,The Kensington Hotel,"[109-113 Queen's Gate, London SW75LR, United K...",None,United Kingdom,105960748767223941294,020 7589 6300,None,51.492500,-0.178217,None,...,None,None,None,None,the kensington hotel,The Kensington Hotel,109 113 Queen s Gate Kensington and Chelsea Lo...,51.4925478,-0.1780611,the kensington hotel
7,The Kensington Hotel,"[29 North Dr, Great Yarmouth, Norfolk NR30 4EW...",Great Yarmouth,United Kingdom,114540475697613464314,01493 844145,None,52.615730,1.737812,None,...,None,None,None,None,the kensington hotel,The Kensington Hotel,109 113 Queen s Gate Kensington and Chelsea Lo...,51.4925478,-0.1780611,the kensington hotel
8,W12 Rooms,"[54 Uxbridge Rd, London W12 8LP, United Kingdom]",London,United Kingdom,107794965009522642601,020 3675 8551,None,51.504445,-0.219223,None,...,None,None,None,None,w12 rooms,W12 Rooms,54 Uxbridge Road Shepherds Bush Hammersmith an...,51.5045326,-0.2191174,w12 rooms
9,The Savoy,"[92 Sandgate, Ayr KA7 1BX, United Kingdom]",None,United Kingdom,112020769094116910189,01292 262563,None,55.462326,-4.634529,None,...,None,None,None,None,the savoy,The Savoy,Strand Westminster Borough London WC2R 0EU Uni...,51.5111922,-0.1194006,the savoy


Step 6. **Fuzzy join**: Fuzzy Join on the normalised hotel name column of the 2 data sets -- google local and distinct hotels in the reviews data set. The fuzzy join is done by considering a Levenshtein distance < 3.  This will produce many false positives.

I perform a second filter step next which compares the latitudes and longitudes. Any false positives will be removed at that stage.

In [18]:
df_google_hotels_norm_levenshtein = google_businesses_df.join(df_distinct_hotels,
                                                             F.levenshtein(
                                                 google_businesses_df.NormalisedHotelName, df_distinct_hotels.Normalised_Hotel_Name) < 3
                                                             )

In [19]:
df_google_hotels_norm_levenshtein.count()

2428

In [22]:
df_google_hotels_norm_levenshtein.limit(10).toPandas()

,name,address,city,country,gPlusPlaceId,phone,price,latitude,longitude,MondayHours,...,ThursdayHours,FridayHours,SaturdayHours,SundayHours,NormalisedHotelName,Hotel_Name,Hotel_Address,lat,lng,Normalised_Hotel_Name
0,St George Hotel,"[49 Gloucester Pl, London W1U 8JE, United King...",London,United Kingdom,105255034750157983625,020 7486 8586,None,51.518083,-0.158144,None,...,None,None,None,None,st george hotel,St George Hotel,49 Gloucester Place Marble Arch Westminster Bo...,51.5182768,-0.1583511,st george hotel
1,Hotel Cavendish,"[75 Gower St, London WC1E 6HJ, United Kingdom]",None,United Kingdom,111392875259801452166,020 7636 9079,None,51.522073,-0.132332,None,...,None,None,None,None,hotel cavendish,Hotel Cavendish,75 Gower Street Camden London WC1E 6HJ United ...,51.5220522,-0.132414,hotel cavendish
2,St George's Hotel,"[St George's Pl, The Promenade, Llandudno, Con...",Llandudno,United Kingdom,113796526104636891443,01492 877544,None,53.324845,-3.828792,None,...,None,None,None,None,st george s hotel,St George Hotel,49 Gloucester Place Marble Arch Westminster Bo...,51.5182768,-0.1583511,st george hotel
3,Hotel Regina,"[Via Regina Levante, 18, 22015 Gravedona ed Un...",None,Italy,105748442507824066289,0344 89446,None,46.148994,9.312078,None,...,None,None,None,None,hotel regina,H tel Regina,2 place des Pyramides 1st arr 75001 Paris France,48.8637503,2.3320406,h tel regina
4,Hyatt Regency Paris Etoile,"[3 Place du Général Kœnig, 75017 Paris, France]",Paris,France,108843403167966082324,01 40 68 12 34,None,48.880255,2.284369,None,...,None,None,None,None,hyatt regency paris etoile,Hyatt Regency Paris Etoile,3 Place du G n ral Koenig 17th arr 75017 Paris...,48.8802782,2.2843852,hyatt regency paris etoile
5,Hotel Regina,"[Piazzale della Puppa, 2, 33081 Piancavallo Po...",None,Italy,108841501360671846117,0434 655166,None,46.107441,12.516762,None,...,None,None,None,None,hotel regina,H tel Regina,2 place des Pyramides 1st arr 75001 Paris France,48.8637503,2.3320406,h tel regina
6,Hotel Regina,"[Avenue Santa Maria, 20260 Calvi, France]",None,France,118039822242888646074,04 95 65 24 23,None,42.564736,8.753747,None,...,None,None,None,None,hotel regina,H tel Regina,2 place des Pyramides 1st arr 75001 Paris France,48.8637503,2.3320406,h tel regina
7,Hotel Regina,"[Gänsackerweg 10, 6534 Serfaus, Austria]",None,Austria,117218391667356161265,05476 6253,None,47.039051,10.606122,None,...,None,None,None,None,hotel regina,H tel Regina,2 place des Pyramides 1st arr 75001 Paris France,48.8637503,2.3320406,h tel regina
8,Hotel Regina,"[Rooseveltplatz 15, 1090 Wien, Austria]",None,Austria,113634606384935765566,01 404460,None,48.216302,16.359817,None,...,None,None,None,None,hotel regina,H tel Regina,2 place des Pyramides 1st arr 75001 Paris France,48.8637503,2.3320406,h tel regina
9,Hotel Regina,"[1 Rue Ropartz Morvan, 29680 Roscoff, France]",None,France,111293295639449237938,02 98 61 23 55,None,48.721077,-3.984033,None,...,None,None,None,None,hotel regina,H tel Regina,2 place des Pyramides 1st arr 75001 Paris France,48.8637503,2.3320406,h tel regina


In [20]:
#df_google_hotels_norm_levenshtein.write.partitionBy("country").format("parquet").save("Data/Cleaned/google_places_levenshtein.parquet")
df_google_hotels_norm_levenshtein.write.format('json').save('Data/Cleaned/google_places_levenshtein.jsonl')

In [92]:
df_google_hotels_norm_levenshtein = spark.read.json('Data/Cleaned/google_places_levenshtein.jsonl')

In [93]:
df_google_hotels_norm_levenshtein.count()

2428

### Data Issues and fixes (apart from the ones already fixed in other notebooks)

Step 7. The plan is to remove false positives by getting the distance between the respective latitude and longitude columns  in the 2 data sets. This is done using the geodesic distance. But before that, we have to fix some issues with these columns. 

In [56]:
df_google_hotels_norm_levenshtein.agg({'latitude': 'max',
                                      'longitude': 'max',
                                      'lat': 'max', 
                                      'lng': 'max'}).collect()

[Row(max(lng)='NA', max(latitude)=55949617.0, max(lat)='NA', max(longitude)=16174364.0)]

In [43]:
df_google_hotels_norm_levenshtein.agg({'latitude': 'min',
                                      'longitude': 'min',
                                      'lat': 'min', 
                                      'lng': 'min'}).collect()

[Row(min(lng)='-0.0023367', min(latitude)=10.682302, min(lat)='41.368437', min(longitude)=-5702343.0)]

We have invalid latitudes and longitudes from the Google Businesses data set. We also seem to have NaNs in the Reviews data set (lat and lng cols). 

Step 8. **Fix latitude and longitude data issue 1: NA values from reviews data set**. We find that there are 16 rows like this, but 10 are for a single hotel, 'Hotel Atlanta'. We drop these rows as they are false positives. For the other 6 rows, we get the values from the lat and lng columns of the Google Local data set and insert it in the latitude and longitude columns of the distinct hotels data set.

In [55]:
df_google_hotels_norm_levenshtein.filter(F.isnan(F.col('lng'))).count()

0

In [26]:
df_google_hotels_norm_levenshtein.filter((df_google_hotels_norm_levenshtein.lat=='NA') |
                                        (df_google_hotels_norm_levenshtein.lng=='NA')).count()

17

In [67]:
df_google_hotels_norm_levenshtein.filter((df_google_hotels_norm_levenshtein.lat=='NA') |
                                        (df_google_hotels_norm_levenshtein.lng=='NA'))\
    .select(['Hotel_Name', 'Hotel_Address', 'name', 'address', 'latitude', 'longitude', 'lat', 'lng']).toPandas()

,Hotel_Name,Hotel_Address,name,address,latitude,longitude,lat,lng
0,Hotel Park Villa,Hasenauerstra e 12 19 D bling 1190 Vienna Austria,Hotel Park-Villa,"[Hasenauerstraße 12, 1190 Wien, Austria]",48.233495,16.345612,NA,NA
1,Hotel Atlanta,W hringer Stra e 33 35 09 Alsergrund 1090 Vien...,Hotel Atlantic,"[Via G. Matteotti, 60, 47043 Gatteo Forlì-Cese...",44.169158,12.437540,NA,NA
2,Hotel Atlanta,W hringer Stra e 33 35 09 Alsergrund 1090 Vien...,Hotel Atlantic,"[Viale Trieste, 365, 61121 Pesaro Pesaro e Urb...",43.918432,12.911133,NA,NA
3,Hotel Atlanta,W hringer Stra e 33 35 09 Alsergrund 1090 Vien...,Hotel Atlantic,"[Corso Repubblica, 124, 28041 Arona Novara, It...",45.756229,8.559663,NA,NA
4,Hotel Atlanta,W hringer Stra e 33 35 09 Alsergrund 1090 Vien...,Hostal Atlanta,"[Carrer Terral, 50, 07600 S'Arenal, ISLAS BALE...",39.499603,2.755165,NA,NA
5,Hotel Atlanta,W hringer Stra e 33 35 09 Alsergrund 1090 Vien...,Hotel Atlanta,"[Neerhem 20, 6301 CH Valkenburg, Netherlands]",50.861029,5.833056,NA,NA
6,Hotel Atlanta,W hringer Stra e 33 35 09 Alsergrund 1090 Vien...,Hotel Atlantic,"[Lungomare Italia, 8, 60017 Senigallia Ancona,...",43.682004,13.279568,NA,NA
7,Hotel Atlanta,W hringer Stra e 33 35 09 Alsergrund 1090 Vien...,HOTEL ATALANTE,"[42 Rue de Genève, 74100 Annemasse, France]",46.193034,6.227661,NA,NA
8,Hotel Atlanta,W hringer Stra e 33 35 09 Alsergrund 1090 Vien...,Hotel Atlanta,"[Calle Alfredo L Jones, 37, 35008 Las Palmas d...",28.142006,-15.433365,NA,NA
9,Cordial Theaterhotel Wien,Josefst dter Stra e 22 08 Josefstadt 1080 Vien...,Cordial Theaterhotel Wien,"[Josefstädter Straße 22, 1080 Wien, Austria]",48.209456,16.351568,NA,NA


There are only 16 of these invalid rows. Also, 10 of these rows are just Hotel Atlanta. These can be fixed manually by comparing the address and adding the latitude and longitude. Let's first drop all the rows with 'Hotel Atlanta', they are all false positives, and we can't filter with geo-coordinates as they are NA.
### Note:
This needs to be done in the original reviews data set too.

In [70]:
df_google_hotels_norm_levenshtein.filter(df_google_hotels_norm_levenshtein.Hotel_Name=='Hotel Atlanta').count()

10

Drop Hotel Atlanta rows

In [94]:
df_google_hotels_norm_levenshtein = df_google_hotels_norm_levenshtein.filter(df_google_hotels_norm_levenshtein.Hotel_Name!='Hotel Atlanta')

In [95]:
df_google_hotels_norm_levenshtein.filter(df_google_hotels_norm_levenshtein.Hotel_Name=='Hotel Atlanta').count()

0

Do the latitude and longitude replacement.

In [73]:
df_google_hotels_norm_levenshtein.filter((df_google_hotels_norm_levenshtein.lat=='NA') |
                                        (df_google_hotels_norm_levenshtein.lng=='NA'))\
    .select(['Hotel_Name', 'Hotel_Address', 'name', 'address', 'latitude', 'longitude', 'lat', 'lng']).toPandas()

,Hotel_Name,Hotel_Address,name,address,latitude,longitude,lat,lng
0,Hotel Park Villa,Hasenauerstra e 12 19 D bling 1190 Vienna Austria,Hotel Park-Villa,"[Hasenauerstraße 12, 1190 Wien, Austria]",48.233495,16.345612,NA,NA
1,Cordial Theaterhotel Wien,Josefst dter Stra e 22 08 Josefstadt 1080 Vien...,Cordial Theaterhotel Wien,"[Josefstädter Straße 22, 1080 Wien, Austria]",48.209456,16.351568,NA,NA
2,Holiday Inn Paris Montmartre,23 Rue Damr mont 18th arr 75018 Paris France,Holiday Inn Paris - Montmartre,"[23 Rue Damrémont, 75018 Paris, France]",48.888800,2.333100,NA,NA
3,Hotel Advance,Sep lveda 180 Eixample 08011 Barcelona Spain,Hotel Advance,"[Carrer Sepúlveda, 180, 08011 Barcelona, Provi...",41.383319,2.162813,NA,NA
4,Renaissance Barcelona Hotel,Pau Clar s 122 Eixample 08009 Barcelona Spain,Renaissance Barcelona Hotel,"[Carrer de Pau Claris, 122, 08009 Barcelona, S...",41.392612,2.167507,NA,NA
5,Roomz Vienna,Paragonstra e 1 11 Simmering 1110 Vienna Austria,Roomz Vienna,"[Paragonstraße 1, 1110 Wien, Austria]",48.186635,16.420513,NA,NA
6,Hotel Daniel Vienna,Landstra er G rtel 5 03 Landstra e 1030 Vienna...,Hotel Daniel Vienna,"[Landstraßer Gürtel 5, 1030 Wien, Austria]",48.188770,16.383972,NA,NA


Replace lat and lng for rows in which they are NA with the corresponding latitude and longitude values respectively.

In [96]:
df_google_hotels_norm_levenshtein = df_google_hotels_norm_levenshtein \
.withColumn('lat_new',F.when(df_google_hotels_norm_levenshtein.lat == 'NA', df_google_hotels_norm_levenshtein.latitude).otherwise(df_google_hotels_norm_levenshtein.lat))\
.drop(df_google_hotels_norm_levenshtein.lat)\
.withColumnRenamed('lat_new', 'lat')

In [97]:
df_google_hotels_norm_levenshtein = df_google_hotels_norm_levenshtein \
.withColumn('lng_new',F.when(df_google_hotels_norm_levenshtein.lng == 'NA', df_google_hotels_norm_levenshtein.longitude).otherwise(df_google_hotels_norm_levenshtein.lng))\
.drop(df_google_hotels_norm_levenshtein.lng)\
.withColumnRenamed('lng_new', 'lng')

In [117]:
df_google_hotels_norm_levenshtein.filter(df_test.Hotel_Name=='Hotel Daniel Vienna').toPandas()

,FridayHours,Hotel_Address,Hotel_Name,MondayHours,NormalisedHotelName,Normalised_Hotel_Name,SaturdayHours,SundayHours,ThursdayHours,TuesdayHours,...,city,country,gPlusPlaceId,latitude,longitude,name,phone,price,lat,lng
0,None,Landstra er G rtel 5 03 Landstra e 1030 Vienna...,Hotel Daniel Vienna,None,hotel daniel vienna,hotel daniel vienna,None,None,None,None,...,None,Austria,114296229584927016602,48.18877,16.383972,Hotel Daniel Vienna,01 901310,None,48.18877,16.383972


In [98]:
df_google_hotels_norm_levenshtein.filter((df_google_hotels_norm_levenshtein.lat=='NA') |
                                        (df_google_hotels_norm_levenshtein.lng=='NA'))\
    .select(['Hotel_Name', 'Hotel_Address', 'name', 'address', 'latitude', 'longitude', 'lat', 'lng']).toPandas()

,Hotel_Name,Hotel_Address,name,address,latitude,longitude,lat,lng


Step 9. **Fix latitude and longitude data issue 2: latitudes and longitudes outside the [-90,90] range**. These values are from the lat and lng columns in the Google Local data set. It seems like all these values are missing a decimal -- they seem to be multiplied by 1,000,000. 

All these rows seem to be false positives (see cells below). We can safely delete them.

In [121]:
df_google_hotels_norm_levenshtein.agg({'latitude': 'max',
                                      'longitude': 'max',
                                      'lat': 'max', 
                                      'lng': 'max'}).collect()

[Row(max(lng)='9.2570459', max(latitude)=55949617.0, max(lat)='52.385601', max(longitude)=16174364.0)]

In [122]:
df_google_hotels_norm_levenshtein.agg({'latitude': 'min',
                                      'longitude': 'min',
                                      'lat': 'min', 
                                      'lng': 'min'}).collect()

[Row(min(lng)='-0.0023367', min(latitude)=10.682302, min(lat)='41.368437', min(longitude)=-5702343.0)]

In [19]:
df_google_hotels_norm_levenshtein.filter((df_google_hotels_norm_levenshtein.latitude < -90 ) |
                                        (df_google_hotels_norm_levenshtein.latitude > 90) |
                                        (df_google_hotels_norm_levenshtein.longitude < -90) |
                                        (df_google_hotels_norm_levenshtein.longitude > 90)).count()

41

So, we have 41 rows with invalid latitude and longitude values. Let's look at these rows.

In [127]:
df_google_hotels_norm_levenshtein.filter((df_google_hotels_norm_levenshtein.latitude < -90 ) |
                                        (df_google_hotels_norm_levenshtein.latitude > 90) |
                                        (df_google_hotels_norm_levenshtein.longitude < -90) |
                                        (df_google_hotels_norm_levenshtein.longitude > 90))\
    .select(['Hotel_Name', 'Hotel_Address', 'name', 'address', 'latitude', 'longitude', 'lat', 'lng'])\
    .toPandas()

,Hotel_Name,Hotel_Address,name,address,latitude,longitude,lat,lng
0,Le A,4 rue d Artois 8th arr 75008 Paris France,Le 24,"[Cours du commerce St André, 59-61 Rue Saint-A...",4.885356e+07,2.338949e+06,48.8723844,2.3088663
1,Arioso,7 rue d Argenson 8th arr 75008 Paris France,Argos,"[Unit 2, Rheidol Retail Park, Aberystwyth, Cer...",5.241303e+02,-4.079567e+01,48.8746066,2.3167195
2,Arioso,7 rue d Argenson 8th arr 75008 Paris France,Argos,"[Unit 2, Jolly Sailor Retail Park, Tamworth, S...",5.262839e+07,-1.708070e+06,48.8746066,2.3167195
3,Arioso,7 rue d Argenson 8th arr 75008 Paris France,Argos,"[30 North St, Sudbury, Suffolk CO10 1RD, Unite...",5.204088e+07,7.299790e+05,48.8746066,2.3167195
4,Arioso,7 rue d Argenson 8th arr 75008 Paris France,Argos,"[4 Berkeley Vale, Falmouth, Cornwall TR11 3PH,...",5.015442e+07,-5.074285e+06,48.8746066,2.3167195
5,Arioso,7 rue d Argenson 8th arr 75008 Paris France,Argos,"[Unit 5, The Broadclose, Peterlee, Co. Durham ...",5.475871e+07,-1.333895e+06,48.8746066,2.3167195
6,Arioso,7 rue d Argenson 8th arr 75008 Paris France,Argos,"[Orchard Centre, 30 Bosley's Orchard, Didcot O...",5.160694e+07,-1.238888e+06,48.8746066,2.3167195
7,41,41 Buckingham Palace Road Westminster Borough ...,Me,"[Carrer de París, 162, 08036 Barcelona, Spain]",4.139201e+07,2.151599e+06,51.4981471,-0.1436492
8,41,41 Buckingham Palace Road Westminster Borough ...,J,"[218 W End Ln, London NW6 1UU, United Kingdom]",5.155272e+07,-1.894860e+05,51.4981471,-0.1436492
9,Cram,Aribau 54 Eixample 08011 Barcelona Spain,Eram,"[CC Alma, 5 Rue du Bosphore, 35200 Rennes, Fra...",4.808312e+07,-1.679306e+06,41.3880549,2.1599602


All these rows seem to be false positives. We can safely delete them.

In [99]:
df_google_hotels_norm_levenshtein = df_google_hotels_norm_levenshtein.filter((df_google_hotels_norm_levenshtein.latitude < 90 ) &
                                        (df_google_hotels_norm_levenshtein.latitude > -90) &
                                        (df_google_hotels_norm_levenshtein.longitude < 90) &
                                        (df_google_hotels_norm_levenshtein.longitude > -90))

In [100]:
df_google_hotels_norm_levenshtein.count()

2377

In [101]:
df_google_hotels_norm_levenshtein.filter((df_google_hotels_norm_levenshtein.latitude < -90 ) |
                                        (df_google_hotels_norm_levenshtein.latitude > 90) |
                                        (df_google_hotels_norm_levenshtein.longitude < -90) |
                                        (df_google_hotels_norm_levenshtein.longitude > 90)).count()

0

Step 10. Now, we are ready to correct the false positives prolem from step 6. We create a new column 'distance' using the latitude and longitude columns (from the reviews data set), and the lat and lng columns (from the google local data set). We do this using the geodesic distance using the geopy library.

In [102]:
@F.udf(returnType=DoubleType())
def calculate_geodesic_distance(lat, lng, latitude, longitude):
    """ Calculates the distance between two sets of coordinates using the Geodesic distance"""
    return distance.distance((lat, lng), (latitude, longitude)).km
    
from geopy import distance
df_google_hotels_distance = df_google_hotels_norm_levenshtein.withColumn(
    'distance', calculate_geodesic_distance('lat', 'lng', 'latitude', 'longitude'))

In [147]:
# Testing
distance.distance(48,2.3,51,-0.9).km

16213.471559599755

In [35]:
df_google_hotels_distance.filter(F.col('distance').isNull()).count()

0

In [163]:
df_google_hotels_distance.limit(10).toPandas()

,FridayHours,Hotel_Address,Hotel_Name,MondayHours,NormalisedHotelName,Normalised_Hotel_Name,SaturdayHours,SundayHours,ThursdayHours,TuesdayHours,...,country,gPlusPlaceId,latitude,longitude,name,phone,price,lat,lng,distance
0,8:00--10:00 pm,4 rue d Artois 8th arr 75008 Paris France,Le A,8:00--10:00 pm,leon,le a,10:00 am--10:00 pm,11:00 am--7:00 pm,8:00--10:00 pm,8:00--10:00 pm,...,United Kingdom,103493159103476742775,51.506552,-0.099553,Leon,020 7620 0035,None,48.8723844,2.3088663,339.711343
1,None,4 rue d Artois 8th arr 75008 Paris France,Le A,None,leo,le a,None,None,None,None,...,France,103937408246271125915,43.693919,7.282358,Leo,None,None,48.8723844,2.3088663,691.213970
2,None,4 rue d Artois 8th arr 75008 Paris France,Le A,None,hema,le a,None,None,None,None,...,Netherlands,107968499121084428167,53.175012,5.418390,HEMA,0517 430 350,None,48.8723844,2.3088663,525.911358
3,None,4 rue d Artois 8th arr 75008 Paris France,Le A,None,lara,le a,None,None,None,None,...,Spain,106999805727842219797,38.435366,-0.840959,Lara,965 47 29 88,None,48.8723844,2.3088663,1186.840223
4,None,4 rue d Artois 8th arr 75008 Paris France,Le A,None,le m,le a,None,None,None,None,...,France,113922546541525201915,48.972683,2.042885,Le M,01 39 27 62 40,None,48.8723844,2.3088663,22.458158
5,None,4 rue d Artois 8th arr 75008 Paris France,Le A,None,ll as,le a,None,None,None,None,...,France,116478106792243581969,48.700800,2.483921,Ll'as,01 69 83 91 37,None,48.8723844,2.3088663,23.012264
6,9:00 am--6:00 pm,4 rue d Artois 8th arr 75008 Paris France,Le A,9:00 am--6:00 pm,hema,le a,9:00 am--6:00 pm,Closed,9:00 am--9:00 pm,9:00 am--6:00 pm,...,Netherlands,106764406647303728656,51.958509,5.897844,Hema,026 323 1663,None,48.8723844,2.3088663,427.577360
7,None,4 rue d Artois 8th arr 75008 Paris France,Le A,None,le jay,le a,None,None,None,None,...,France,105647946029563375724,46.962910,2.921518,Le Jay,02 48 74 18 53,None,48.8723844,2.3088663,217.192027
8,None,4 rue d Artois 8th arr 75008 Paris France,Le A,None,hema,le a,None,None,None,None,...,Netherlands,109900589358081101370,51.847259,5.863737,HEMA,024 322 0557,None,48.8723844,2.3088663,416.417559
9,None,4 rue d Artois 8th arr 75008 Paris France,Le A,None,hema,le a,None,None,None,None,...,Netherlands,116396317554914703807,52.385932,6.276086,HEMA,0572 351 273,None,48.8723844,2.3088663,481.045952


We can look at all hotels which have a distance 10 between their data in the 2 data sets.Exploring the distance values showed that most of the true positives have distance < 0.3 between the respective latitudes and longitudes from the two data sets.

In [172]:
df_google_hotels_distance.filter(F.col('distance') < 10)\
.select(['Hotel_Name', 'Hotel_Address', 'name', 'address', 'distance', 'latitude', 'longitude', 'lat', 'lng'])\
.toPandas().to_csv('distance_test.csv')

In [103]:
df_google_hotels_distance = df_google_hotels_distance.filter(F.col('distance') < 0.58)

In [25]:
#df_google_hotels_distance = df_google_hotels_distance.filter(F.col('distance') < 0.3)

In [ ]:
#df_google_hotels_distance.filter(~F.col('distance').isin([0.427064259369519, 0.308453755411568, 0.219268485234889])).count()

In [104]:
df_google_hotels_distance = df_google_hotels_distance.filter(~F.col('distance').isin([0.427064259369519, 0.308453755411568, 0.219268485234889]))

In [105]:
df_google_hotels_distance.count()

462

Check for and drop duplicates

In [49]:
df_google_hotels_distance \
    .groupby(['Hotel_Name', 'lat', 'lng']) \
    .count() \
    .where('count > 1') \
    .sort('count', ascending=False) \
    .limit(8).toPandas()

,Hotel_Name,lat,lng,count
0,Relais Du Louvre,48.8592264,2.340996,2
1,Ayre Hotel Rosell n,41.4047382,2.1727127,2
2,Hotel Berna,45.4826692,9.2034371,2
3,Austria Trend Hotel Europa Wien,48.2062103,16.3710387,2


In [53]:
#df_google_hotels_distance.exceptAll(df.drop_duplicates(['ID', 'ID2', 'Number'])).show()
dup_hotels = ['Austria Trend Hotel Europa Wien', 'Hotel Berna', 'Ayre Hotel Rosell n', 'Relais Du Louvre']
df_google_hotels_distance.select(['Hotel_Name', 'address', 'Hotel_Address', 'name']).filter(F.col('Hotel_Name').isin(dup_hotels)).toPandas()

,Hotel_Name,address,Hotel_Address,name
0,Hotel Berna,"[Via Napo Torriani, 27, 20124 Milano, Italy]",Via Napo Torriani 18 Central Station 20124 Mil...,Hotel Bernina
1,Hotel Berna,"[Via Napo Torriani, 18, 20124 Milano, Italy]",Via Napo Torriani 18 Central Station 20124 Mil...,Hotel Berna
2,Austria Trend Hotel Europa Wien,"[Kärntner Straße 32, 1010 Wien, Austria]",K rntnerstra e 18 Eingang Neuer Markt 3 01 Inn...,Austria Trend Hotel Europa Wien
3,Austria Trend Hotel Europa Wien,"[Kärntner Straße 18, 1010 Wien, Austria]",K rntnerstra e 18 Eingang Neuer Markt 3 01 Inn...,Austria Trend Hotel Europa Wien
4,Relais Du Louvre,"[3 Rue du Louvre, 75001 Paris, France]",19 Rue Des Pretres Saint Germain L Auxerrois 1...,Relais Du Louvre
5,Relais Du Louvre,"[19 Rue des Prêtres Saint-Germain l'Auxerrois,...",19 Rue Des Pretres Saint Germain L Auxerrois 1...,Relais Du Louvre
6,Ayre Hotel Rosell n,"[Carrer del Rossellón, 390, 08025 Barcelona, S...",Rossell 390 Eixample 08025 Barcelona Spain,Ayre Hotel Rosellon
7,Ayre Hotel Rosell n,"[Carrer del Rossellón, 390, 08025 Barcelona, S...",Rossell 390 Eixample 08025 Barcelona Spain,Ayre Hotel Rosellon


In [59]:
df_google_hotels_distance.filter(~((F.col('name') == 'Hotel Bernina') & (F.col('Hotel_Name')=='Hotel Berna'))).count()

461

In [106]:
df_google_hotels_distance = df_google_hotels_distance.filter(~((F.col('name') == 'Hotel Bernina') & (F.col('Hotel_Name')=='Hotel Berna')))
df_google_hotels_distance = df_google_hotels_distance.dropDuplicates(subset=['Hotel_Name', 'lat', 'lng'])
df_google_hotels_distance.count()

458

Step 12. As we don't have all hotels from the reviews data set in the google local data set, we need the remaining hotels as well from the reviews data set. To get these hotels, we perform a set difference using the Hotel_Name column in the data sets (data set in Step 1 and data set in Step 11). Remember that this column in Step 11 originally came from Step 1, so it will have the same values for the rows which were successfully joined.

In [107]:
# Remove the invalid row -- it has no location
df_distinct_hotels = df_distinct_hotels.filter(F.col('Hotel_Name')!= 'Hotel Atlanta')
df_distinct_hotels.count()

1493

In [33]:
df_distinct_hotels.limit(3).toPandas()

,Hotel_Name,Hotel_Address,lat,lng,Normalised_Hotel_Name
0,Hilton London Paddington,146 Praed Street Westminster Borough London W2...,51.515632,-0.175737,hilton london paddington
1,Millennium Hotel London Knightsbridge,17 25 Sloane Street Knightsbridge Kensington a...,51.4999683,-0.1599927,millennium hotel london knightsbridge
2,Grand Pigalle Hotel,29 rue Victor Mass 9th arr 75009 Paris France,48.880856,2.3375085,grand pigalle hotel


In [41]:
df_distinct_hotels.filter(
    (F.col('lat').isNull() | (F.col('lng').isNull()))).count()

0

A left anti-join will get the hotels in df_distinct_hotels which are not in df_google_hotels_distance. One possible solution would be to get these hotels and then do a union. 

However, a much simpler solution would be do to a left join between these 2 data frames.

In [108]:
df_distinct_hotels.join(df_google_hotels_distance, on=['Hotel_Name', 'lat', 'lng'], how='left').count()

1493

As we are going to join the df_google_hotels_distance with the original df_distinct_hotels table (where a few of its columns came from), it is imperative to drop these columns at this stage. Otherwise, we will have duplicate columns when the same column name exists in the two data frames to be joined. 

Also, we drop the other unnecessary columns

In [109]:
columns_to_drop = ['Hotel_Address', 'Normalised_Hotel_Name', 'distance', 'latitude', 'longitude']
df_google_hotels_distance = df_google_hotels_distance.drop(*columns_to_drop)
df_google_hotels_distance.columns

['FridayHours',
 'Hotel_Name',
 'MondayHours',
 'NormalisedHotelName',
 'SaturdayHours',
 'SundayHours',
 'ThursdayHours',
 'TuesdayHours',
 'WednesdayHours',
 'address',
 'city',
 'country',
 'gPlusPlaceId',
 'name',
 'phone',
 'price',
 'lat',
 'lng']

In [110]:
df_google_hotels = df_distinct_hotels.join(df_google_hotels_distance, on=['Hotel_Name', 'lat', 'lng'], how='left')

In [111]:
df_google_hotels.count()

1493

Step 13. **Data issue: the hotel name and address in the reviews data set does not render latin characters properly**. Use the hotel name from the google local data set, where available to populate a new hotel name column. When it is not available, use the hotel name from the reviews data set.

In [112]:
df_google_hotels = df_google_hotels.withColumn('HotelName', F.when(~F.col('name').isNull(), F.col('name')).otherwise(F.col('Hotel_Name')))
print(df_google_hotels.count())
df_google_hotels.limit(10).toPandas()


1493


,Hotel_Name,lat,lng,Hotel_Address,Normalised_Hotel_Name,FridayHours,MondayHours,NormalisedHotelName,SaturdayHours,SundayHours,...,TuesdayHours,WednesdayHours,address,city,country,gPlusPlaceId,name,phone,price,HotelName
0,Hilton London Paddington,51.515632,-0.175737,146 Praed Street Westminster Borough London W2...,hilton london paddington,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,Hilton London Paddington
1,Millennium Hotel London Knightsbridge,51.4999683,-0.1599927,17 25 Sloane Street Knightsbridge Kensington a...,millennium hotel london knightsbridge,None,None,millennium hotel london knightsbridge,None,None,...,None,None,"[17 Sloane St, London SW1X 9NU, United Kingdom]",None,United Kingdom,102371155121784386535,Millennium Hotel London Knightsbridge,020 7235 4377,None,Millennium Hotel London Knightsbridge
2,Grand Pigalle Hotel,48.880856,2.3375085,29 rue Victor Mass 9th arr 75009 Paris France,grand pigalle hotel,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,Grand Pigalle Hotel
3,St George Hotel,51.5182768,-0.1583511,49 Gloucester Place Marble Arch Westminster Bo...,st george hotel,None,None,st george hotel,None,None,...,None,None,"[49 Gloucester Pl, London W1U 8JE, United King...",London,United Kingdom,105255034750157983625,St George Hotel,020 7486 8586,None,St George Hotel
4,H tel Regent s Garden,48.8797513,2.2919634,6 Rue Pierre Demours 17th arr 75017 Paris France,h tel regent s garden,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,H tel Regent s Garden
5,Hotel Cavendish,51.5220522,-0.132414,75 Gower Street Camden London WC1E 6HJ United ...,hotel cavendish,None,None,hotel cavendish,None,None,...,None,None,"[75 Gower St, London WC1E 6HJ, United Kingdom]",None,United Kingdom,111392875259801452166,Hotel Cavendish,020 7636 9079,None,Hotel Cavendish
6,Best Western PLUS Epping Forest,51.603207,0.010607,Epping Forest 30 Oak Hill London IG8 9NY Unite...,best western plus epping forest,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,Best Western PLUS Epping Forest
7,Fleming s Selection Hotel Wien City,NA,NA,Josefst dter Stra e 10 12 08 Josefstadt 1080 V...,fleming s selection hotel wien city,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,Fleming s Selection Hotel Wien City
8,Eurostars Bcn Design,41.3907208,2.1660732,Passeig de Gr cia 29 Eixample 08007 Barcelona ...,eurostars bcn design,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,Eurostars Bcn Design
9,The Yard Milano,45.453152,9.179172,Piazza XXIV Maggio 8 Milan City Center 20123 M...,the yard milano,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,The Yard Milano


Step 14. Get the country from the hotel address string column.

In [113]:
@F.udf(returnType=StringType())
def get_country(address, countries_list=['france', 'italy', 'spain', 'united kingdom', 'austria', 'netherlands']):
    """ Gets the country from the hotel address"""
    #geo_address = GeoText(address.title())
    matching_country = [country for country in countries_list if country in address.lower()][0].title()
    return matching_country

df_google_hotels = df_google_hotels.withColumn('Country', get_country('Hotel_Address'))

In [114]:
df_google_hotels.select('Country').distinct().show()

+--------------+
|       Country|
+--------------+
|        France|
|         Italy|
|         Spain|
|       Austria|
|United Kingdom|
|   Netherlands|
+--------------+



In [115]:
# Check for null Country
df_google_hotels.filter(F.col('Country').isNull()).count()

0

Step 15. Remove all adiditional columns which won't be written to the staging table: latitude, longitude, distance the second hotel name, second hotel address. 
Step 16. Rename the columns as required.

In [234]:
df_google_hotels.columns

['Hotel_Name',
 'lat',
 'lng',
 'HotelAddress',
 'FridayHours',
 'MondayHours',
 'NormalisedHotelName',
 'SaturdayHours',
 'SundayHours',
 'ThursdayHours',
 'TuesdayHours',
 'WednesdayHours',
 'GoogleAddress',
 'Country',
 'gPlusPlaceId',
 'Phone',
 'Price',
 'HotelName',
 'HotelID']

In [116]:
columns_to_drop = ['name', 'Normalised_Hotel_Name', 'Normalised_Hotel_Name', 'city'] 
df_google_hotels = df_google_hotels.drop(*columns_to_drop)
df_google_hotels.columns

['Hotel_Name',
 'lat',
 'lng',
 'Hotel_Address',
 'FridayHours',
 'MondayHours',
 'NormalisedHotelName',
 'SaturdayHours',
 'SundayHours',
 'ThursdayHours',
 'TuesdayHours',
 'WednesdayHours',
 'address',
 'Country',
 'gPlusPlaceId',
 'phone',
 'price',
 'HotelName']

In [117]:
df_google_hotels = df_google_hotels.withColumnRenamed('address', 'GoogleAddress')
df_google_hotels = df_google_hotels.withColumnRenamed('Hotel_Address', 'HotelAddress')
df_google_hotels = df_google_hotels.withColumnRenamed('phone', 'Phone')
df_google_hotels = df_google_hotels.withColumnRenamed('price', 'Price')


In [118]:
df_google_hotels.columns

['Hotel_Name',
 'lat',
 'lng',
 'HotelAddress',
 'FridayHours',
 'MondayHours',
 'NormalisedHotelName',
 'SaturdayHours',
 'SundayHours',
 'ThursdayHours',
 'TuesdayHours',
 'WednesdayHours',
 'GoogleAddress',
 'Country',
 'gPlusPlaceId',
 'Phone',
 'Price',
 'HotelName']

Step 17. **Assign a new primary key column to the hotels: HotelID**

In [119]:
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window
# lit('A') is just dummy data so that we can calculate row number over the complete data set,
# we don't have separate groups/windows of data.
w = Window().orderBy(lit('A'))
#df_google_hotels.withColumn("HotelID", row_number().over(w)).select('HotelID').agg({'HotelID': 'max'}).show()
df_google_hotels = df_google_hotels.withColumn("HotelID", row_number().over(w))

Step 18. Data Fix: Manually fill in the latitude and longitude of 16 hotels for which they are both 'NA'. I get these values from Google (manually) as there are only 16 rows.

In [233]:
df_google_hotels.filter((F.col('lat')=='NA')| (F.col('lng')=='NA')).toPandas()

,Hotel_Name,lat,lng,HotelAddress,FridayHours,MondayHours,NormalisedHotelName,SaturdayHours,SundayHours,ThursdayHours,TuesdayHours,WednesdayHours,GoogleAddress,Country,gPlusPlaceId,Phone,Price,HotelName,HotelID
0,Fleming s Selection Hotel Wien City,NA,NA,Josefst dter Stra e 10 12 08 Josefstadt 1080 V...,None,None,None,None,None,None,None,None,None,Austria,None,None,None,Fleming s Selection Hotel Wien City,8
1,Austria Trend Hotel Schloss Wilhelminenberg Wien,NA,NA,Savoyenstra e 2 16 Ottakring 1160 Vienna Austria,None,None,None,None,None,None,None,None,None,Austria,None,None,None,Austria Trend Hotel Schloss Wilhelminenberg Wien,11
2,Hotel Park Villa,NA,NA,Hasenauerstra e 12 19 D bling 1190 Vienna Austria,None,None,None,None,None,None,None,None,None,Austria,None,None,None,Hotel Park Villa,182
3,Hotel Advance,NA,NA,Sep lveda 180 Eixample 08011 Barcelona Spain,None,None,None,None,None,None,None,None,None,Spain,None,None,None,Hotel Advance,277
4,Roomz Vienna,NA,NA,Paragonstra e 1 11 Simmering 1110 Vienna Austria,None,None,None,None,None,None,None,None,None,Austria,None,None,None,Roomz Vienna,545
5,NH Collection Barcelona Podium,NA,NA,Bail n 4 6 Eixample 08010 Barcelona Spain,None,None,None,None,None,None,None,None,None,Spain,None,None,None,NH Collection Barcelona Podium,587
6,City Hotel Deutschmeister,NA,NA,Gr nentorgasse 30 09 Alsergrund 1090 Vienna Au...,None,None,None,None,None,None,None,None,None,Austria,None,None,None,City Hotel Deutschmeister,905
7,Hotel Daniel Vienna,NA,NA,Landstra er G rtel 5 03 Landstra e 1030 Vienna...,None,None,None,None,None,None,None,None,None,Austria,None,None,None,Hotel Daniel Vienna,958
8,Derag Livinghotel Kaiser Franz Joseph Vienna,NA,NA,Sieveringer Stra e 4 19 D bling 1190 Vienna Au...,None,None,None,None,None,None,None,None,None,Austria,None,None,None,Derag Livinghotel Kaiser Franz Joseph Vienna,1122
9,Hotel City Central,NA,NA,Taborstra e 8 A 02 Leopoldstadt 1020 Vienna Au...,None,None,None,None,None,None,None,None,None,Austria,None,None,None,Hotel City Central,1124


In [238]:
def replace_na_with_latlong(df, HotelID, latit, longi):
    df = df.withColumn('lat', F.when(F.col('HotelID')==HotelID, latit).otherwise(F.col('lat')))
    df = df.withColumn('lng', F.when(F.col('HotelID')==HotelID, longi).otherwise(F.col('lng')))
    return df
    
df_google_hotels = replace_na_with_latlong(df_google_hotels, 8, 48.2094, 16.3534) # Fleming s Selection Hotel Wien City
df_google_hotels = replace_na_with_latlong(df_google_hotels, 11, 48.2196, 16.2856) # Austria Trend Hotel Schloss Wilhelminenberg Wien
df_google_hotels = replace_na_with_latlong(df_google_hotels, 182, 48.2336, 16.3457) # Hotel Park Villa
df_google_hotels = replace_na_with_latlong(df_google_hotels, 277, 41.3833, 2.1628) # Hotel Advance
df_google_hotels = replace_na_with_latlong(df_google_hotels, 545, 48.1868, 16.4207) # Roomz Vienna
df_google_hotels = replace_na_with_latlong(df_google_hotels, 587, 41.3916, 2.1779) # NH Collection Barcelona Podium
df_google_hotels = replace_na_with_latlong(df_google_hotels, 905, 48.2209, 16.3666) # City Hotel Deutschmeister
df_google_hotels = replace_na_with_latlong(df_google_hotels, 958, 48.1888, 16.3838) # Hotel Daniel Vienna
df_google_hotels = replace_na_with_latlong(df_google_hotels, 1122, 48.2460, 16.3415) # Derag Livinghotel Kaiser Franz Joseph Vienna
df_google_hotels = replace_na_with_latlong(df_google_hotels, 1124, 48.2136, 16.3799) # Hotel City Central
df_google_hotels = replace_na_with_latlong(df_google_hotels, 1125, 48.2168, 16.3601) # Hotel Pension Baron am Schottentor
df_google_hotels = replace_na_with_latlong(df_google_hotels, 1316, 48.8400, 2.3236) # Mercure Paris Gare Montparnasse
df_google_hotels = replace_na_with_latlong(df_google_hotels, 1366, 48.8889, 2.3331) # Holiday Inn Paris Montmartre
df_google_hotels = replace_na_with_latlong(df_google_hotels, 1372, 48.2095, 16.3515) # Cordial Theaterhotel Wien
df_google_hotels = replace_na_with_latlong(df_google_hotels, 1472, 41.3927, 2.1674) # Renaissance Barcelona Hotel


In [241]:
df_google_hotels.filter(F.col('HotelID')==8).toPandas()

,Hotel_Name,lat,lng,HotelAddress,FridayHours,MondayHours,NormalisedHotelName,SaturdayHours,SundayHours,ThursdayHours,TuesdayHours,WednesdayHours,GoogleAddress,Country,gPlusPlaceId,Phone,Price,HotelName,HotelID
0,Fleming s Selection Hotel Wien City,48.2094,16.3534,Josefst dter Stra e 10 12 08 Josefstadt 1080 V...,None,None,None,None,None,None,None,None,None,Austria,None,None,None,Fleming s Selection Hotel Wien City,8


In [239]:
df_google_hotels.filter((F.col('lat')=='NA')| (F.col('lng')=='NA')).toPandas()

,Hotel_Name,lat,lng,HotelAddress,FridayHours,MondayHours,NormalisedHotelName,SaturdayHours,SundayHours,ThursdayHours,TuesdayHours,WednesdayHours,GoogleAddress,Country,gPlusPlaceId,Phone,Price,HotelName,HotelID
0,Maison Albar Hotel Paris Op ra Diamond,NA,NA,4 rue de la P pini re 8th arr 75008 Paris France,None,None,None,None,None,None,None,None,None,France,None,None,None,Maison Albar Hotel Paris Op ra Diamond,1355


In [240]:
df_google_hotels = replace_na_with_latlong(df_google_hotels, 1355,48.8753, 2.3234) # Maison Albar Hotel Paris Op ra Diamond

df_google_hotels.filter((F.col('lat')=='NA')| (F.col('lng')=='NA')).toPandas()

,Hotel_Name,lat,lng,HotelAddress,FridayHours,MondayHours,NormalisedHotelName,SaturdayHours,SundayHours,ThursdayHours,TuesdayHours,WednesdayHours,GoogleAddress,Country,gPlusPlaceId,Phone,Price,HotelName,HotelID


## Part 2: Get the nearest airport to each hotel from Airport codes, get the final version of the Hotels and Airports data frames

Step 19. Read in airport_codes and filter so that we have airports in the EU. Also, **Data issue: remove spam records**

In [191]:
df_airport_codes = spark.read.csv('Data/Original/airport-codes_csv.csv', header=True)
df_airport_codes.count()

55075

There are spam records in the airport codes data set for some reason. These need to be thrown out. They all seem to have coordinates = 0, 0.

In [192]:
df_airport_codes.filter(F.col('coordinates')=='0, 0').limit(5).toPandas()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,AE-0027,closed,marked as spam,None,AS,AE,AE-U-A,None,0000,None,None,"0, 0"
1,AE-0028,small_airport,osman anisa,None,AS,AE,AE-DU,Business Bay,None,None,None,"0, 0"
2,AE-0030,medium_airport,FIVE STAR FINANCE COMPANY,None,AS,AE,AE-DU,None,None,None,None,"0, 0"
3,AL-0004,medium_airport,Çá¸¾á¸á¸ á¸®á¸Ç{+91-9680118734} GiRLFRieNd...,None,EU,AL,AL-U-A,ki,None,None,None,"0, 0"
4,AU-0116,small_airport,SYD,None,OC,AU,AU-NSW,None,None,None,None,"0, 0"


In [193]:
df_airport_codes.filter(F.col('coordinates')=='0, 0').count()

53

In [194]:
df_airport_codes = df_airport_codes.filter(F.col('coordinates') != '0, 0')

In [195]:
df_airport_codes = df_airport_codes.filter(F.col('type')!='closed')
df_airport_codes.count()

51425

Restrict to EU

In [196]:
df_airport_codes = df_airport_codes.filter(F.col('continent')=='EU')
df_airport_codes.count()

6965

In [197]:
df_airport_codes.select('type').distinct().show()

+--------------+
|          type|
+--------------+
| large_airport|
|   balloonport|
| seaplane_base|
|      heliport|
|medium_airport|
| small_airport|
+--------------+



We are only interested in small, medium, large airports

In [198]:
df_airport_codes = df_airport_codes.filter(F.col('type').isin('small_airport', 'medium_airport', 'large_airport'))
df_airport_codes.count()

5910

In [199]:
df_airport_codes.select('iso_country').distinct().show()

+-----------+
|iso_country|
+-----------+
|         LT|
|         FI|
|         UA|
|         RO|
|         NL|
|         PL|
|         MK|
|         EE|
|         SM|
|         AT|
|         RU|
|         HR|
|         CZ|
|         PT|
|         GG|
|         GB|
|         MT|
|         BY|
|         MD|
|         DE|
+-----------+
only showing top 20 rows



Split the coordinates string column into 2 columns: latitude and longitude. Bizzarely, the airport codes data contains the longitude before the latitude.

In [203]:
# https://stackoverflow.com/questions/39235704/split-spark-dataframe-string-column-into-multiple-columns
split_col = F.split(df_airport_codes['coordinates'], ', ')
df_airport_codes = df_airport_codes.withColumn('longitude', split_col.getItem(0))
df_airport_codes = df_airport_codes.withColumn('latitude', split_col.getItem(1))

In [204]:
df_airport_codes = df_airport_codes.withColumn('longitude', df_airport_codes.longitude.cast(DoubleType()))
df_airport_codes = df_airport_codes.withColumn('latitude', df_airport_codes.latitude.cast(DoubleType()))
print(df_airport_codes.dtypes)
df_airport_codes.limit(5).toPandas()

[('ident', 'string'), ('type', 'string'), ('name', 'string'), ('elevation_ft', 'string'), ('continent', 'string'), ('iso_country', 'string'), ('iso_region', 'string'), ('municipality', 'string'), ('gps_code', 'string'), ('iata_code', 'string'), ('local_code', 'string'), ('coordinates', 'string'), ('longitude', 'double'), ('latitude', 'double')]


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,longitude,latitude
0,BA-0001,small_airport,Sport airfield Ciljuge,725,EU,BA,BA-BIH,Tuzla,LQCG,None,None,"18.685613632202, 44.438152313232",18.685614,44.438152
1,BA-0002,small_airport,Sportski Aerodrom ZaluÅ¾ani,None,EU,BA,BA-BIH,ZaluÅ¾ani,LQBZ,None,None,"17.222826, 44.8482",17.222826,44.848200
2,BA-0003,small_airport,Brod Airstrip,337,EU,BA,BA-SRP,Brod,None,None,None,"18.761888, 44.846235",18.761888,44.846235
3,BE-0003,small_airport,Sovet Airfield,919,EU,BE,BE-WNA,Ciney,None,None,None,"5.04027795791, 50.28666687",5.040278,50.286667
4,BE-0006,small_airport,[Duplicate] Neerpelt ULM,148,EU,BE,BE-VLI,None,EBNE,None,None,"5.478611, 51.211944",5.478611,51.211944


In [205]:
df_airport_codes = df_airport_codes.drop('coordinates')

Step 20. Add in the country based on the countryiso.csv ISO-Country name mapping file.

In [206]:
isocountries_df = spark.read.csv('Data/Original/countryiso.csv', header=True)
isocountries_df.count()

249

In [207]:
isocountries_df = isocountries_df.withColumnRenamed('Name', 'Country')
isocountries_df.limit(3).toPandas()

,Country,Code
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL


In [208]:
df_airport_codes = df_airport_codes.join(isocountries_df, df_airport_codes.iso_country==isocountries_df.Code)

In [226]:
df_airport_codes.filter((F.col('latitude')=='NA')| (F.col('longitude')=='NA')).count()

0

In [228]:
df_airport_codes.filter((F.isnan('latitude'))| (F.isnan('longitude'))).count()

0

In [209]:
df_airport_codes.limit(3).toPandas()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,longitude,latitude,Country,Code
0,BA-0001,small_airport,Sport airfield Ciljuge,725,EU,BA,BA-BIH,Tuzla,LQCG,None,None,18.685614,44.438152,Bosnia and Herzegovina,BA
1,BA-0002,small_airport,Sportski Aerodrom ZaluÅ¾ani,None,EU,BA,BA-BIH,ZaluÅ¾ani,LQBZ,None,None,17.222826,44.848200,Bosnia and Herzegovina,BA
2,BA-0003,small_airport,Brod Airstrip,337,EU,BA,BA-SRP,Brod,None,None,None,18.761888,44.846235,Bosnia and Herzegovina,BA


In [210]:
df_airport_codes = df_airport_codes.withColumnRenamed('ident', 'AirportID')
df_airport_codes = df_airport_codes.withColumnRenamed('name', 'AirportName')

In [211]:
df_airport_codes = df_airport_codes.drop('Code')

In [212]:
df_airport_codes.limit(3).toPandas()

,AirportID,type,AirportName,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,longitude,latitude,Country
0,BA-0001,small_airport,Sport airfield Ciljuge,725,EU,BA,BA-BIH,Tuzla,LQCG,None,None,18.685614,44.438152,Bosnia and Herzegovina
1,BA-0002,small_airport,Sportski Aerodrom ZaluÅ¾ani,None,EU,BA,BA-BIH,ZaluÅ¾ani,LQBZ,None,None,17.222826,44.848200,Bosnia and Herzegovina
2,BA-0003,small_airport,Brod Airstrip,337,EU,BA,BA-SRP,Brod,None,None,None,18.761888,44.846235,Bosnia and Herzegovina


Step 21. Insert the nearest airport to every hotel Now, we need to join the airport codes data with the df_google_hotels data based on Country. We only want to insert the 'AirportID', 'latitude' and 'longitude' columns into the df_google_hotels data frame along with all the existing columns. Obviously, we will get a lot of duplicate data as every row in a particular country will join with every airport in that country.

In [175]:
df_google_hotels.join(df_airport_codes, on='Country')\
    .select(df_google_hotels['*'], df_airport_codes.AirportID, df_airport_codes.latitude, df_airport_codes.longitude)\
    .count()

697118

In [242]:
df_google_hotels_with_dups = df_google_hotels.join(df_airport_codes, on='Country')\
    .select(df_google_hotels['*'], df_airport_codes.AirportID, df_airport_codes.latitude, df_airport_codes.longitude)

In [243]:
df_google_hotels_with_dups.limit(5).toPandas()

,Hotel_Name,lat,lng,HotelAddress,FridayHours,MondayHours,NormalisedHotelName,SaturdayHours,SundayHours,ThursdayHours,...,GoogleAddress,Country,gPlusPlaceId,Phone,Price,HotelName,HotelID,AirportID,latitude,longitude
0,Grand Pigalle Hotel,48.880856,2.3375085,29 rue Victor Mass 9th arr 75009 Paris France,None,None,None,None,None,None,...,None,France,None,None,None,Grand Pigalle Hotel,3,FR-0098,43.194168,3.051667
1,Grand Pigalle Hotel,48.880856,2.3375085,29 rue Victor Mass 9th arr 75009 Paris France,None,None,None,None,None,None,...,None,France,None,None,None,Grand Pigalle Hotel,3,FR-0158,46.890148,-2.088156
2,Grand Pigalle Hotel,48.880856,2.3375085,29 rue Victor Mass 9th arr 75009 Paris France,None,None,None,None,None,None,...,None,France,None,None,None,Grand Pigalle Hotel,3,FR-0183,43.082699,5.933806
3,Grand Pigalle Hotel,48.880856,2.3375085,29 rue Victor Mass 9th arr 75009 Paris France,None,None,None,None,None,None,...,None,France,None,None,None,Grand Pigalle Hotel,3,FR-0242,45.376718,6.276920
4,Grand Pigalle Hotel,48.880856,2.3375085,29 rue Victor Mass 9th arr 75009 Paris France,None,None,None,None,None,None,...,None,France,None,None,None,Grand Pigalle Hotel,3,FR-0246,42.900002,1.275000


Step 22. Calculate the geodesic distance using exactly the same method as before

In [245]:
@F.udf(returnType=DoubleType())
def calculate_geodesic_distance(lat, lng, latitude, longitude):
    """ Calculates the distance between two sets of coordinates using the Geodesic distance"""
    return distance.distance((lat, lng), (latitude, longitude)).km
    
from geopy import distance
df_google_hotels_with_dups = df_google_hotels_with_dups.withColumn(
    'distance', calculate_geodesic_distance('lat', 'lng', 'latitude', 'longitude'))

In [246]:
df_google_hotels_with_dups.limit(5).toPandas()

,Hotel_Name,lat,lng,HotelAddress,FridayHours,MondayHours,NormalisedHotelName,SaturdayHours,SundayHours,ThursdayHours,...,Country,gPlusPlaceId,Phone,Price,HotelName,HotelID,AirportID,latitude,longitude,distance
0,Grand Pigalle Hotel,48.880856,2.3375085,29 rue Victor Mass 9th arr 75009 Paris France,None,None,None,None,None,None,...,France,None,None,None,Grand Pigalle Hotel,3,FR-0098,43.194168,3.051667,634.491510
1,Grand Pigalle Hotel,48.880856,2.3375085,29 rue Victor Mass 9th arr 75009 Paris France,None,None,None,None,None,None,...,France,None,None,None,Grand Pigalle Hotel,3,FR-0158,46.890148,-2.088156,398.083062
2,Grand Pigalle Hotel,48.880856,2.3375085,29 rue Victor Mass 9th arr 75009 Paris France,None,None,None,None,None,None,...,France,None,None,None,Grand Pigalle Hotel,3,FR-0183,43.082699,5.933806,701.931381
3,Grand Pigalle Hotel,48.880856,2.3375085,29 rue Victor Mass 9th arr 75009 Paris France,None,None,None,None,None,None,...,France,None,None,None,Grand Pigalle Hotel,3,FR-0242,45.376718,6.276920,490.873303
4,Grand Pigalle Hotel,48.880856,2.3375085,29 rue Victor Mass 9th arr 75009 Paris France,None,None,None,None,None,None,...,France,None,None,None,Grand Pigalle Hotel,3,FR-0246,42.900002,1.275000,669.843161


In [237]:
df_google_hotels_with_dups.dtypes

[('Hotel_Name', 'string'),
 ('lat', 'string'),
 ('lng', 'string'),
 ('HotelAddress', 'string'),
 ('FridayHours', 'string'),
 ('MondayHours', 'string'),
 ('NormalisedHotelName', 'string'),
 ('SaturdayHours', 'string'),
 ('SundayHours', 'string'),
 ('ThursdayHours', 'string'),
 ('TuesdayHours', 'string'),
 ('WednesdayHours', 'string'),
 ('GoogleAddress', 'array<string>'),
 ('Country', 'string'),
 ('gPlusPlaceId', 'string'),
 ('Phone', 'string'),
 ('Price', 'string'),
 ('HotelName', 'string'),
 ('HotelID', 'int'),
 ('AirportID', 'string'),
 ('latitude', 'double'),
 ('longitude', 'double'),
 ('distance', 'double')]

In [244]:
# Testing.
df_google_hotels_with_dups.filter((F.col('lat')=='NA')| (F.col('lng')=='NA')).count()

0

Manual test: check the min distance airport from HotelID=3. Then check whether this is the airport which is the min distance airport.

In [247]:
df_google_hotels_with_dups.filter(df_google_hotels_with_dups.HotelID==3).agg({'distance': 'min'}).show()

+------------------+
|     min(distance)|
+------------------+
|12.446208126422647|
+------------------+



In [248]:
df_google_hotels_with_dups.filter(F.col('distance')==12.446208126422647).show()

+-------------------+---------+---------+--------------------+-----------+-----------+-------------------+-------------+-----------+-------------+------------+--------------+-------------+-------+------------+-----+-----+-------------------+-------+---------+------------------+-----------------+------------------+
|         Hotel_Name|      lat|      lng|        HotelAddress|FridayHours|MondayHours|NormalisedHotelName|SaturdayHours|SundayHours|ThursdayHours|TuesdayHours|WednesdayHours|GoogleAddress|Country|gPlusPlaceId|Phone|Price|          HotelName|HotelID|AirportID|          latitude|        longitude|          distance|
+-------------------+---------+---------+--------------------+-----------+-----------+-------------------+-------------+-----------+-------------+------------+--------------+-------------+-------+------------+-----+-----+-------------------+-------+---------+------------------+-----------------+------------------+
|Grand Pigalle Hotel|48.880856|2.3375085|29 rue Vict

In [249]:
df_airport_codes.filter(F.col('AirportID')=='LFPB').toPandas()

,AirportID,type,AirportName,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,longitude,latitude,Country
0,LFPB,medium_airport,Paris-Le Bourget Airport,218,EU,FR,FR-J,Paris,LFPB,LBG,None,2.44139,48.969398,France


Step 23. Use a Window function by partitioning over the HotelID column. This will be used to get the minimum distance over each window. The airport with the minimum distance from the hotel is the nearest airport. This is a two-step process. First, we create a new column min_distance which contains the minimum distance to an airport for a window. Then, we filter rows where min_distance=distance to get the final results.

In [250]:
from pyspark.sql.window import Window
w = Window.partitionBy("HotelID")

In [260]:
del df_test
df_google_hotels_with_dups = df_google_hotels_with_dups.select(*df_google_hotels_with_dups.columns, F.min('distance').over(w).alias('min_distance'))

In [261]:
df_google_hotels_with_dups.limit(10).toPandas()

,Hotel_Name,lat,lng,HotelAddress,FridayHours,MondayHours,NormalisedHotelName,SaturdayHours,SundayHours,ThursdayHours,...,gPlusPlaceId,Phone,Price,HotelName,HotelID,AirportID,latitude,longitude,distance,min_distance
0,Courtyard by Marriott Amsterdam Arena Atlas,52.3101182,4.9472918,Hoogoorddreef 1 Zuidoost 1101 BA Amsterdam Net...,None,None,None,None,None,None,...,None,None,None,Courtyard by Marriott Amsterdam Arena Atlas,148,EHAL,53.451698,5.677220,136.210443,11.187869
1,Courtyard by Marriott Amsterdam Arena Atlas,52.3101182,4.9472918,Hoogoorddreef 1 Zuidoost 1101 BA Amsterdam Net...,None,None,None,None,None,None,...,None,None,None,Courtyard by Marriott Amsterdam Arena Atlas,148,EHAM,52.308601,4.763890,12.509798,11.187869
2,Courtyard by Marriott Amsterdam Arena Atlas,52.3101182,4.9472918,Hoogoorddreef 1 Zuidoost 1101 BA Amsterdam Net...,None,None,None,None,None,None,...,None,None,None,Courtyard by Marriott Amsterdam Arena Atlas,148,EHBD,51.255299,5.601390,125.742567,11.187869
3,Courtyard by Marriott Amsterdam Arena Atlas,52.3101182,4.9472918,Hoogoorddreef 1 Zuidoost 1101 BA Amsterdam Net...,None,None,None,None,None,None,...,None,None,None,Courtyard by Marriott Amsterdam Arena Atlas,148,EHBK,50.911701,5.770140,165.697380,11.187869
4,Courtyard by Marriott Amsterdam Arena Atlas,52.3101182,4.9472918,Hoogoorddreef 1 Zuidoost 1101 BA Amsterdam Net...,None,None,None,None,None,None,...,None,None,None,Courtyard by Marriott Amsterdam Arena Atlas,148,EHDB,52.099541,5.176417,28.184708,11.187869
5,Courtyard by Marriott Amsterdam Arena Atlas,52.3101182,4.9472918,Hoogoorddreef 1 Zuidoost 1101 BA Amsterdam Net...,None,None,None,None,None,None,...,None,None,None,Courtyard by Marriott Amsterdam Arena Atlas,148,EHDL,52.060600,5.873060,69.136057,11.187869
6,Courtyard by Marriott Amsterdam Arena Atlas,52.3101182,4.9472918,Hoogoorddreef 1 Zuidoost 1101 BA Amsterdam Net...,None,None,None,None,None,None,...,None,None,None,Courtyard by Marriott Amsterdam Arena Atlas,148,EHDP,51.517300,5.855720,108.113708,11.187869
7,Courtyard by Marriott Amsterdam Arena Atlas,52.3101182,4.9472918,Hoogoorddreef 1 Zuidoost 1101 BA Amsterdam Net...,None,None,None,None,None,None,...,None,None,None,Courtyard by Marriott Amsterdam Arena Atlas,148,EHDR,53.119202,6.129720,120.377646,11.187869
8,Courtyard by Marriott Amsterdam Arena Atlas,52.3101182,4.9472918,Hoogoorddreef 1 Zuidoost 1101 BA Amsterdam Net...,None,None,None,None,None,None,...,None,None,None,Courtyard by Marriott Amsterdam Arena Atlas,148,EHDS,52.468333,6.333333,95.989474,11.187869
9,Courtyard by Marriott Amsterdam Arena Atlas,52.3101182,4.9472918,Hoogoorddreef 1 Zuidoost 1101 BA Amsterdam Net...,None,None,None,None,None,None,...,None,None,None,Courtyard by Marriott Amsterdam Arena Atlas,148,EHEH,51.450100,5.374530,100.110056,11.187869


In [256]:
df_google_hotels_with_dups.filter(df_google_hotels_with_dups.HotelID==148).agg({'distance': 'min'}).show()

+------------------+
|     min(distance)|
+------------------+
|11.187869335386422|
+------------------+



Filter rows where the min_distance == distance, those are the rows we want to keep

In [263]:
df_google_hotels_with_dups.filter(F.col('distance') == F.col('min_distance')).count()

1493

In [265]:
df_google_hotels_with_airportid = df_google_hotels_with_dups.filter(F.col('distance') == F.col('min_distance'))

In [266]:
df_google_hotels_with_airportid.count()

1493

In [268]:
df_google_hotels_with_airportid.limit(10).toPandas()

,Hotel_Name,lat,lng,HotelAddress,FridayHours,MondayHours,NormalisedHotelName,SaturdayHours,SundayHours,ThursdayHours,...,gPlusPlaceId,Phone,Price,HotelName,HotelID,AirportID,latitude,longitude,distance,min_distance
0,Courtyard by Marriott Amsterdam Arena Atlas,52.3101182,4.9472918,Hoogoorddreef 1 Zuidoost 1101 BA Amsterdam Net...,None,None,None,None,None,None,...,None,None,None,Courtyard by Marriott Amsterdam Arena Atlas,148,NL-0050,52.281602,4.790039,11.187869,11.187869
1,Hotel The Serras,41.3801815,2.180688,Passeig de Colom 9 Ciutat Vella 08002 Barcelon...,None,None,None,None,None,None,...,None,None,None,Hotel The Serras,463,LEBL,41.297100,2.078460,12.584021,12.584021
2,Hotel Maison Ath n e,48.8713421,2.3280723,19 Rue De Caumartin 9th arr 75009 Paris France,None,None,None,None,None,None,...,None,None,None,Hotel Maison Ath n e,471,LFPB,48.969398,2.441390,13.707033,13.707033
3,Maison Albar H tel Paris Champs Elys es ex Mac...,48.8753837,2.2944651,3 avenue Mac Mahon 17th arr 75017 Paris France,None,None,None,None,None,None,...,None,None,None,Maison Albar H tel Paris Champs Elys es ex Mac...,496,LFPV,48.774167,2.191667,13.552623,13.552623
4,Catalonia Magdalenes,41.3861283,2.1745291,Magdalenes 13 15 Ciutat Vella 08002 Barcelona ...,None,None,None,None,None,None,...,None,None,None,Catalonia Magdalenes,833,LEBL,41.297100,2.078460,12.744391,12.744391
5,Mokinba Hotels Montebianco,45.4788051,9.1446276,Via Monte Rosa 90 Fiera Milano City 20149 Mila...,None,None,None,None,None,None,...,None,None,None,Mokinba Hotels Montebianco,1088,LIMB,45.542198,9.203330,8.407316,8.407316
6,ARCOTEL Kaiserwasser Superior,48.231915,16.417026,Wagramer Stra e 8 22 Donaustadt 1220 Vienna Au...,None,None,None,None,None,None,...,None,None,None,ARCOTEL Kaiserwasser Superior,1238,LOWW,48.110298,16.569700,17.658495,17.658495
7,L Edmond H tel,48.8823111,2.3132374,22 avenue de Villiers 17th arr 75017 Paris France,None,None,l edmond hotel,None,None,None,...,107909375352052892577,01 44 01 09 40,None,L'edmond Hotel,1342,LFPB,48.969398,2.441390,13.490302,13.490302
8,Palais Hansen Kempinski Vienna,48.2163149,16.3685103,Schottenring 24 01 Innere Stadt 1010 Vienna Au...,None,None,palais hansen kempinski vienna,None,None,None,...,115300235437179621025,01 2361000,None,Palais Hansen Kempinski Vienna,243,LOWW,48.110298,16.569700,19.051488,19.051488
9,Hotel Ares Eiffel,48.8503276,2.2982022,7 rue du G n ral de Larminat 15th arr 75015 Pa...,None,None,hotel ares eiffel,None,None,None,...,117469042429892205022,01 47 34 74 04,None,Hotel Ares Eiffel,392,LFPV,48.774167,2.191667,11.530764,11.530764


Step 24. Create a lookup data frame with columns hotel_id, Hotel_name (from original data frame), lat and lng. This will be used to assign Hotel IDs as FKs to the original reviews data set in Part 3.

In [273]:
df_hotelid_lookup = df_google_hotels_with_airportid.select('HotelID', 'Hotel_Name', 'HotelName', 'lat', 'lng')

In [270]:
df_hotelid_lookup.count()

1493

In [274]:
df_hotelid_lookup.limit(5).toPandas()

,HotelID,Hotel_Name,HotelName,lat,lng
0,148,Courtyard by Marriott Amsterdam Arena Atlas,Courtyard by Marriott Amsterdam Arena Atlas,52.3101182,4.9472918
1,463,Hotel The Serras,Hotel The Serras,41.3801815,2.180688
2,471,Hotel Maison Ath n e,Hotel Maison Ath n e,48.8713421,2.3280723
3,496,Maison Albar H tel Paris Champs Elys es ex Mac...,Maison Albar H tel Paris Champs Elys es ex Mac...,48.8753837,2.2944651
4,833,Catalonia Magdalenes,Catalonia Magdalenes,41.3861283,2.1745291


Step 25. Create final version of the clean df_google_hotels_with_airport_id data frame. In this step, we drop unnecessary columns, rename and reorder columns. This is saved in JSONL format, will be loaded into S3 in Part 4, and will be loaded into a Redshift staging table from there.

In [275]:
columns_to_drop = ['latitude', 'longitude', 'distance', 'min_distance', 'NormalisedHotelName']
df_google_hotels_with_airportid = df_google_hotels_with_airportid.drop(*columns_to_drop)
df_google_hotels_with_airportid.columns

['Hotel_Name',
 'lat',
 'lng',
 'HotelAddress',
 'FridayHours',
 'MondayHours',
 'SaturdayHours',
 'SundayHours',
 'ThursdayHours',
 'TuesdayHours',
 'WednesdayHours',
 'GoogleAddress',
 'Country',
 'gPlusPlaceId',
 'Phone',
 'Price',
 'HotelName',
 'HotelID',
 'AirportID']

In [277]:
df_google_hotels_with_airportid = df_google_hotels_with_airportid.withColumnRenamed('lat', 'Latitude')
df_google_hotels_with_airportid = df_google_hotels_with_airportid.withColumnRenamed('lng', 'Longitude')
df_google_hotels_with_airportid = df_google_hotels_with_airportid.withColumnRenamed('AirportID', 'NearestAirportID')
df_google_hotels_with_airportid = df_google_hotels_with_airportid.withColumnRenamed('Hotel_Name', 'OriginalHotelName')
df_google_hotels_with_airportid.columns

['OriginalHotelName',
 'Latitude',
 'Longitude',
 'HotelAddress',
 'FridayHours',
 'MondayHours',
 'SaturdayHours',
 'SundayHours',
 'ThursdayHours',
 'TuesdayHours',
 'WednesdayHours',
 'GoogleAddress',
 'Country',
 'gPlusPlaceId',
 'Phone',
 'Price',
 'HotelName',
 'HotelID',
 'NearestAirportID']

In [278]:
column_order = ['HotelID', 'HotelName', 'HotelAddress', 'Country', 'Phone', 'Price',
                'OriginalHotelName', 'GoogleAddress',  'Latitude', 'Longitude', 'gPlusPlaceId',
                'NearestAirportID', 'MondayHours', 'TuesdayHours', 'WednesdayHours',
                'ThursdayHours', 'FridayHours', 'SaturdayHours',  'SundayHours']
df_google_hotels_with_airportid = df_google_hotels_with_airportid.select(*column_order)
df_google_hotels_with_airportid.columns

['HotelID',
 'HotelName',
 'HotelAddress',
 'Country',
 'Phone',
 'Price',
 'OriginalHotelName',
 'GoogleAddress',
 'Latitude',
 'Longitude',
 'gPlusPlaceId',
 'NearestAirportID',
 'MondayHours',
 'TuesdayHours',
 'WednesdayHours',
 'ThursdayHours',
 'FridayHours',
 'SaturdayHours',
 'SundayHours']

In [279]:
df_google_hotels_with_airportid.limit(5).toPandas()

,HotelID,HotelName,HotelAddress,Country,Phone,Price,OriginalHotelName,GoogleAddress,Latitude,Longitude,gPlusPlaceId,NearestAirportID,MondayHours,TuesdayHours,WednesdayHours,ThursdayHours,FridayHours,SaturdayHours,SundayHours
0,148,Courtyard by Marriott Amsterdam Arena Atlas,Hoogoorddreef 1 Zuidoost 1101 BA Amsterdam Net...,Netherlands,None,None,Courtyard by Marriott Amsterdam Arena Atlas,None,52.3101182,4.9472918,None,NL-0050,None,None,None,None,None,None,None
1,463,Hotel The Serras,Passeig de Colom 9 Ciutat Vella 08002 Barcelon...,Spain,None,None,Hotel The Serras,None,41.3801815,2.180688,None,LEBL,None,None,None,None,None,None,None
2,471,Hotel Maison Ath n e,19 Rue De Caumartin 9th arr 75009 Paris France,France,None,None,Hotel Maison Ath n e,None,48.8713421,2.3280723,None,LFPB,None,None,None,None,None,None,None
3,496,Maison Albar H tel Paris Champs Elys es ex Mac...,3 avenue Mac Mahon 17th arr 75017 Paris France,France,None,None,Maison Albar H tel Paris Champs Elys es ex Mac...,None,48.8753837,2.2944651,None,LFPV,None,None,None,None,None,None,None
4,833,Catalonia Magdalenes,Magdalenes 13 15 Ciutat Vella 08002 Barcelona ...,Spain,None,None,Catalonia Magdalenes,None,41.3861283,2.1745291,None,LEBL,None,None,None,None,None,None,None


Step 26. Create final version of the df_airport_codes data frame. In this step, we drop unnecessary columns, rename and reorder columns. This is saved in JSONL format, will be loaded into S3 in Part 4, and will be directly loaded into a Redshift dimension table from there. 

In [280]:
df_airport_codes.columns

['AirportID',
 'type',
 'AirportName',
 'elevation_ft',
 'continent',
 'iso_country',
 'iso_region',
 'municipality',
 'gps_code',
 'iata_code',
 'local_code',
 'longitude',
 'latitude',
 'Country']

In [281]:
df_airport_codes = df_airport_codes.withColumnRenamed('type', 'Type')
df_airport_codes = df_airport_codes.withColumnRenamed('elevation_ft', 'ElevationInFeet')
df_airport_codes = df_airport_codes.withColumnRenamed('continent', 'Continent')
df_airport_codes = df_airport_codes.withColumnRenamed('iso_country', 'ISOCountry')
df_airport_codes = df_airport_codes.withColumnRenamed('iso_region', 'ISORegion')
df_airport_codes = df_airport_codes.withColumnRenamed('municipality', 'Municipality')
df_airport_codes = df_airport_codes.withColumnRenamed('gps_code', 'GPSCode')
df_airport_codes = df_airport_codes.withColumnRenamed('iata_code', 'IATACode')
df_airport_codes = df_airport_codes.withColumnRenamed('local_code', 'LocalCode')
df_airport_codes = df_airport_codes.withColumnRenamed('longitude', 'Longitude')
df_airport_codes = df_airport_codes.withColumnRenamed('latitude', 'Latitude')
df_airport_codes.columns

['AirportID',
 'Type',
 'AirportName',
 'ElevationInFeet',
 'Continent',
 'ISOCountry',
 'ISORegion',
 'Municipality',
 'GPSCode',
 'IATACode',
 'LocalCode',
 'Longitude',
 'Latitude',
 'Country']

In [302]:
column_order = ['AirportID', 'Type', 'AirportName', 'Municipality', 'Country', 'ISOCountry',
                'Continent', 'ISORegion', 'Latitude', 'Longitude', 'ElevationInFeet',
                'GPSCode', 'IATACode', 'LocalCode']
df_airport_codes = df_airport_codes.select(*column_order)
df_airport_codes.columns

['AirportID',
 'Type',
 'AirportName',
 'Municipality',
 'Country',
 'ISOCountry',
 'Continent',
 'ISORegion',
 'Latitude',
 'Longitude',
 'ElevationInFeet',
 'GPSCode',
 'IATACode',
 'LocalCode']

Checkpoint: save the airports and hotels results. Write locally.

In [284]:
df_google_hotels_with_airportid.write.format('json').save('Data/Cleaned/hotels.jsonl')

In [304]:
df_google_hotels_with_airportid.write.partitionBy('Country').format('parquet').save('Data/Cleaned/hotels.parquet')

In [285]:
df_airport_codes.count()

5908

In [307]:
df_airport_codes.filter(F.col('municipality')=='NA').count()

0

In [14]:
df_airport_codes = df_airport_codes.filter(~F.col('AirportName').contains('Duplicate'))

Step 26b. **Data issue** : Some rows contain a string which starts with 'Duplicate' for the hotel name. These are duplicate rows and can be removed.

In [15]:
df_airport_codes.count()

5878

In [16]:
df_airport_codes.write.format('json').mode('overwrite').save('Data/Cleaned/airports.jsonl')

In [295]:
df_airport_codes.write.format('csv').mode('overwrite').save('Data/Cleaned/airports.csv', header="True")

In [303]:
df_airport_codes.write.partitionBy('Country').format('parquet').save('Data/Cleaned/airports.parquet')

In [305]:
df_hotelid_lookup.drop('HotelName').write.format('parquet').save('Data/Cleaned/hotelidlookup.parquet')

In [10]:
df_airport_codes = spark.read.parquet('Data/Cleaned/airports.parquet')

In [6]:
df_google_hotels_with_airportid = spark.read.parquet('Data/Cleaned/hotels.parquet')

In [92]:
df_hotelid_lookup = spark.read.parquet('Data/Cleaned/hotelidlookup.parquet')

## Part 3: Assign Hotel IDs from the created data frames to the Reviews data set (foreign keys), create the final version of the reviews data frame.

In [93]:
df_hotel_reviews.limit(5).toPandas()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.3605759,4.9159683
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.3605759,4.9159683
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.3605759,4.9159683
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.3605759,4.9159683
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.3605759,4.9159683


In [94]:
df_hotel_reviews.printSchema()

root
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: string (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: string (nullable = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)



Step 27. Our first goal is to join the Hotelid_lookup data frame with the reviews data frame on Hotel_Name, lat and lng. However, we need to fix the same issues we fixed while building the data frame for the join to take place successfully. In other words, we need to do something about the rows which have 'NA' lat/long values.
Perform the same process as Step 18. Drop rows with Hotel_Name='Hotel Atlanta'. Add in the latitude and longitude for the rows with NA, i.e., for the same 16 hotels that were manipulated in Step 18.


In [95]:
df_hotel_reviews = df_hotel_reviews.filter(df_hotel_reviews.Hotel_Name!='Hotel Atlanta')

In [96]:
df_hotel_reviews.count()

515349

In [36]:
df_hotel_reviews.filter((F.col('lat')=='NA') | (F.col('lng')=='NA')).count()

2879

Check if there are multiple hotels with the names of the hotels below (the names are from Step 18). This is checked in the following cell using dropDuplicates. Once all the checks are done, the  next cell is executed to add the latitude and longitude of these cells.

In [97]:
def replace_na_with_latlong(df, hotelname, latit, longi):
    df = df.withColumn('lat', F.when(F.col('Hotel_Name')==hotelname, latit).otherwise(F.col('lat')))
    df = df.withColumn('lng', F.when(F.col('Hotel_Name')==hotelname, longi).otherwise(F.col('lng')))
    return df
    
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'Fleming s Selection Hotel Wien City', 48.2094, 16.3534) # Fleming s Selection Hotel Wien City
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'Austria Trend Hotel Schloss Wilhelminenberg Wien', 48.2196, 16.2856) # Austria Trend Hotel Schloss Wilhelminenberg Wien
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'Hotel Park Villa', 48.2336, 16.3457) # Hotel Park Villa
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'Hotel Advance', 41.3833, 2.1628) # Hotel Advance
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'Roomz Vienna', 48.1868, 16.4207) # Roomz Vienna
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'NH Collection Barcelona Podium', 41.3916, 2.1779) # NH Collection Barcelona Podium
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'City Hotel Deutschmeister', 48.2209, 16.3666) # City Hotel Deutschmeister
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'Hotel Daniel Vienna', 48.1888, 16.3838) # Hotel Daniel Vienna
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'Derag Livinghotel Kaiser Franz Joseph Vienna', 48.2460, 16.3415) # Derag Livinghotel Kaiser Franz Joseph Vienna
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'Hotel City Central', 48.2136, 16.3799) # Hotel City Central
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'Hotel Pension Baron am Schottentor', 48.2168, 16.3601) # Hotel Pension Baron am Schottentor
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'Mercure Paris Gare Montparnasse', 48.8400, 2.3236) # Mercure Paris Gare Montparnasse
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'Holiday Inn Paris Montmartre', 48.8889, 2.3331) # Holiday Inn Paris Montmartre
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'Cordial Theaterhotel Wien', 48.2095, 16.3515) # Cordial Theaterhotel Wien
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'Renaissance Barcelona Hotel', 41.3927, 2.1674) # Renaissance Barcelona Hotel
df_hotel_reviews = replace_na_with_latlong(df_hotel_reviews, 'Maison Albar Hotel Paris Op ra Diamond', 48.8753, 2.3234) # Maison Albar Hotel Paris Op ra Diamond


In [98]:
df_hotel_reviews.filter(F.col('Hotel_Name')=='Cordial Theaterhotel Wien').dropDuplicates(subset=['Hotel_Name', 'Hotel_Address']).count()

1

In [99]:
df_hotel_reviews.filter((F.col('lat')=='NA') | (F.col('lng')=='NA')).count()

0

Step 28. Join the df_hotelid_lookup data frame with the reviews data frame on (Hotel_Name, lat, lng).

In [100]:
df_hotel_reviews.join(df_hotelid_lookup, on=['Hotel_Name', 'lat', 'lng']).dropDuplicates(subset=['HotelID']).count()

1493

In [101]:
df_hotel_reviews = df_hotel_reviews.join(df_hotelid_lookup, on=['Hotel_Name', 'lat', 'lng'])

In [102]:
df_hotel_reviews.limit(5).toPandas()

,Hotel_Name,lat,lng,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,HotelID
0,Hotel Arena,52.3605759,4.9159683,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,298
1,Hotel Arena,52.3605759,4.9159683,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,298
2,Hotel Arena,52.3605759,4.9159683,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,298
3,Hotel Arena,52.3605759,4.9159683,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,298
4,Hotel Arena,52.3605759,4.9159683,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,298


Step 29. As we have the HotelId now in our data frame, we can drop all the other information like Hotel Name, lat, lng and Hotel Address. These are already identified by the Hotel ID. Also, the days since review is a useless field and can be removed.

In [103]:
df_hotel_reviews = df_hotel_reviews.drop(*['Hotel_Name', 'lat', 'Hotel_Address', 'lng', 'days_since_review'])

In [104]:
df_hotel_reviews.printSchema()

root
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: string (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: string (nullable = true)
 |-- HotelID: integer (nullable = true)



Step 30. As we saw in the Explore_reviews notebook, the string columns have leading and spaces. These need to be stripped. Also, some string columns have empty spaces as their values. These values need to be replaced with None -- we want a Null to be added into the final database.

In [105]:
@F.udf(returnType=StringType())
def strip_leading_trailing_spaces(val):
    return val.strip()
string_columns = ['Review_Date', 'Reviewer_Nationality', 'Negative_Review', 'Positive_Review', 'Tags']
df_hotel_reviews.select([F.count(F.when(strip_leading_trailing_spaces(c)=='', c)).alias(c) for c in string_columns]).toPandas()

,Review_Date,Reviewer_Nationality,Negative_Review,Positive_Review,Tags
0,0,522,847,183,0


In [106]:
# Replace the values in the string columns with the stripped values.
for c in string_columns:
    df_hotel_reviews= df_hotel_reviews.withColumn(c, strip_leading_trailing_spaces(c))

In [107]:
def blank_as_null(val):
    return F.when(F.col(val) != "", F.col(val)).otherwise(None)

for c in string_columns:
    df_hotel_reviews= df_hotel_reviews.withColumn(c, blank_as_null(c))

Check if the empty string values have been turned into Nulls

In [108]:
df_hotel_reviews.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in string_columns]).toPandas()

,Review_Date,Reviewer_Nationality,Negative_Review,Positive_Review,Tags
0,0,522,847,183,0


Step 31. Change the datatype of the Review_Date field from string to date.

In [109]:
df_hotel_reviews = df_hotel_reviews.withColumn('Review_Date', F.to_date(F.col('Review_Date'), format='MM/dd/yyyy'))

In [110]:
df_hotel_reviews.limit(10).toPandas()

,Additional_Number_of_Scoring,Review_Date,Average_Score,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,HotelID
0,194,2017-08-03,7.7,Russia,I am so angry that i made this post available ...,397,1403,Only the park outside of the hotel was beautiful,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",298
1,194,2017-08-03,7.7,Ireland,No Negative,0,1403,No real complaints the hotel was great great l...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",298
2,194,2017-07-31,7.7,Australia,Rooms are nice but for elderly a bit difficult...,42,1403,Location was good and staff were ok It is cute...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",298
3,194,2017-07-31,7.7,United Kingdom,My room was dirty and I was afraid to walk bar...,210,1403,Great location in nice surroundings the bar an...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",298
4,194,2017-07-24,7.7,New Zealand,You When I booked with your company on line yo...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",298
5,194,2017-07-24,7.7,Poland,Backyard of the hotel is total mess shouldn t ...,17,1403,Good restaurant with modern design great chill...,20,1,6.7,"[' Leisure trip ', ' Group ', ' Duplex Double ...",298
6,194,2017-07-17,7.7,United Kingdom,Cleaner did not change our sheet and duvet eve...,33,1403,The room is spacious and bright The hotel is l...,18,6,4.6,"[' Leisure trip ', ' Group ', ' Duplex Twin Ro...",298
7,194,2017-07-17,7.7,United Kingdom,Apart from the price for the brekfast Everythi...,11,1403,Good location Set in a lovely park friendly st...,19,1,10.0,"[' Leisure trip ', ' Couple ', ' Duplex Double...",298
8,194,2017-07-09,7.7,Belgium,Even though the pictures show very clean rooms...,34,1403,No Positive,0,3,6.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",298
9,194,2017-07-08,7.7,Norway,The aircondition makes so much noise and its h...,15,1403,The room was big enough and the bed is good Th...,50,1,7.9,"[' Leisure trip ', ' Couple ', ' Large King Ro...",298


In [81]:
df_hotel_reviews.filter(df_hotel_reviews.Review_Date.isNull()).count()

0

In [82]:
df_hotel_reviews.printSchema()

root
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: date (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: string (nullable = true)
 |-- HotelID: integer (nullable = true)



Step 32. Create the final version of the df_hotel_reviews data frame. We  rename and reorder columns, and finally sort the data frame by date. This will be loaded into S3 in Part 4, and will be loaded into a Redshift staging table from there.

In [111]:
df_hotel_reviews = df_hotel_reviews.withColumnRenamed('Additional_Number_of_Scoring', 'NumRatings')
df_hotel_reviews = df_hotel_reviews.withColumnRenamed('Review_Date', 'ReviewDate')
df_hotel_reviews = df_hotel_reviews.withColumnRenamed('Average_Score', 'AverageScore')
df_hotel_reviews = df_hotel_reviews.withColumnRenamed('Reviewer_Nationality', 'ReviewerNationality')
df_hotel_reviews = df_hotel_reviews.withColumnRenamed('Negative_Review', 'NegativeReview')
df_hotel_reviews = df_hotel_reviews.withColumnRenamed('Review_Total_Negative_Word_Counts', 'ReviewNegativeWordsCount')
df_hotel_reviews = df_hotel_reviews.withColumnRenamed('Total_Number_of_Reviews', 'NumReviews')
df_hotel_reviews = df_hotel_reviews.withColumnRenamed('Positive_Review', 'PositiveReview')
df_hotel_reviews = df_hotel_reviews.withColumnRenamed('Review_Total_Positive_Word_Counts', 'ReviewPositiveWordsCount')
df_hotel_reviews = df_hotel_reviews.withColumnRenamed('Total_Number_of_Reviews_Reviewer_Has_Given', 'TotalNumReviewsByReviewer')
df_hotel_reviews = df_hotel_reviews.withColumnRenamed('Reviewer_Score', 'ReviewerScore')

In [112]:
df_hotel_reviews.columns

['NumRatings',
 'ReviewDate',
 'AverageScore',
 'ReviewerNationality',
 'NegativeReview',
 'ReviewNegativeWordsCount',
 'NumReviews',
 'PositiveReview',
 'ReviewPositiveWordsCount',
 'TotalNumReviewsByReviewer',
 'ReviewerScore',
 'Tags',
 'HotelID']

In [113]:
column_order = ['HotelID', 'NumReviews', 'AverageScore', 'NumRatings', 'ReviewDate', 
                'ReviewerNationality', 'TotalNumReviewsByReviewer', 'ReviewerScore',
                'NegativeReview', 'ReviewNegativeWordsCount', 'PositiveReview',
                'ReviewPositiveWordsCount', 'Tags']
df_hotel_reviews = df_hotel_reviews.select(*column_order)
df_hotel_reviews.columns

['HotelID',
 'NumReviews',
 'AverageScore',
 'NumRatings',
 'ReviewDate',
 'ReviewerNationality',
 'TotalNumReviewsByReviewer',
 'ReviewerScore',
 'NegativeReview',
 'ReviewNegativeWordsCount',
 'PositiveReview',
 'ReviewPositiveWordsCount',
 'Tags']

In [114]:
df_hotel_reviews.count()

515349

In [115]:
df_hotel_reviews = df_hotel_reviews.sort(df_hotel_reviews.ReviewDate.asc())

In [116]:
df_hotel_reviews.limit(5).toPandas()

,HotelID,NumReviews,AverageScore,NumRatings,ReviewDate,ReviewerNationality,TotalNumReviewsByReviewer,ReviewerScore,NegativeReview,ReviewNegativeWordsCount,PositiveReview,ReviewPositiveWordsCount,Tags
0,173,2619,9.2,639,2015-08-04,United Kingdom,3,9.6,Nothing not to like second time we have stayed...,12,Excellent service we arrived at 11 30 am and w...,27,"[' Leisure trip ', ' Couple ', ' City King Roo..."
1,173,2619,9.2,639,2015-08-04,United Kingdom,3,10.0,Breakfast could have been better organised wit...,45,We were visiting London to celebrate our 25th ...,29,"[' Leisure trip ', ' Couple ', ' City King Roo..."
2,173,2619,9.2,639,2015-08-04,Switzerland,3,8.8,The room on the first day was a mile away from...,33,Good sized rooms Staff friendly and very helpful,10,"[' Leisure trip ', ' Couple ', ' City Twin Roo..."
3,298,1403,7.7,194,2015-08-04,United Kingdom,1,9.2,No Negative,0,The staff were so friendly and very helpful,10,"[' Leisure trip ', ' Couple ', ' Duplex Double..."
4,298,1403,7.7,194,2015-08-04,United Kingdom,1,9.2,No Negative,0,The hotel is amazing beautiful old building an...,15,"[' Leisure trip ', ' Group ', ' Duplex Double ..."


Step 33. Write the reviews dataframe to disk in JSONL format.

In [120]:
df_hotel_reviews.write.json('Data/Cleaned/reviews.jsonl')

In [ ]:
#df_hotel_reviews.write.mode('overwrite').partitionBy(['ReviewDate', 'HotelID']).parquet('Data/Cleaned/reviews.parquet')

Step 34. Read in the created CountryIndicators.csv File (created in the **CombineAdditionalCountryData notebook**), convert it into JSONL format, and write it into a local directory.

In [5]:
df_country_indicators = spark.read.csv('Data/Cleaned/CountryIndicators.csv', header=True)

In [45]:
df_country_indicators.limit(3).toPandas()

,_c0,Country,ISOCode,TourismExpenditureMillions,TouristArrivalsThousands,Currency,ExchangeRateEndOfPeriod,GNIPerCapita,GDPPerCapita,MobilePhoneSubscriptions,...,UrbanPopulationPercent,HDIRank,HDI,InternetUsersPercent,PoliticalRightsFreedomScore,CivilLibertiesFreedomScore,FreedomStatus,DemocracyOrNot,PoliticalRegimeTypeScore,HumanRightsScore
0,0,Afghanistan,AF,50.0,None,Afghani (AFN),74.9556,1746.0,1735.0,59.1,...,25.5,170.0,0.496,None,5.0,6.0,Not Free,False,-1.0,-2.209409986
1,1,Åland Islands,AX,None,None,Euro (EUR),0.8734,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,2,Albania,AL,2306.0,5340.0,Lek (ALL),107.82,12300.0,12306.0,94.2,...,60.3,69.0,0.7909999999999999,None,3.0,3.0,Partly Free,True,9.0,0.7703089679999999


The first column is just a row number. We can drop it.

In [6]:
df_country_indicators = df_country_indicators.drop('_c0')
df_country_indicators.limit(3).toPandas()

,Country,ISOCode,TourismExpenditureMillions,TouristArrivalsThousands,Currency,ExchangeRateEndOfPeriod,GNIPerCapita,GDPPerCapita,MobilePhoneSubscriptions,NetMigrationRate,...,UrbanPopulationPercent,HDIRank,HDI,InternetUsersPercent,PoliticalRightsFreedomScore,CivilLibertiesFreedomScore,FreedomStatus,DemocracyOrNot,PoliticalRegimeTypeScore,HumanRightsScore
0,Afghanistan,AF,50.0,None,Afghani (AFN),74.9556,1746.0,1735.0,59.1,-1.7,...,25.5,170.0,0.496,None,5.0,6.0,Not Free,False,-1.0,-2.209409986
1,Åland Islands,AX,None,None,Euro (EUR),0.8734,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Albania,AL,2306.0,5340.0,Lek (ALL),107.82,12300.0,12306.0,94.2,-4.9,...,60.3,69.0,0.7909999999999999,None,3.0,3.0,Partly Free,True,9.0,0.7703089679999999


In [21]:
df_country_indicators.filter(F.col('Country').isNull()).count()

0

In [17]:
df_country_indicators.columns

['Country',
 'ISOCode',
 'TourismExpenditureMillions',
 'TouristArrivalsThousands',
 'Currency',
 'ExchangeRateEndOfPeriod',
 'GNIPerCapita',
 'GDPPerCapita',
 'MobilePhoneSubscriptions',
 'NetMigrationRate',
 'Population',
 'UrbanPopulationPercent',
 'HDIRank',
 'HDI',
 'InternetUsersPercent',
 'PoliticalRightsFreedomScore',
 'CivilLibertiesFreedomScore',
 'FreedomStatus',
 'DemocracyOrNot',
 'PoliticalRegimeTypeScore',
 'HumanRightsScore']

In [47]:
df_country_indicators.write.parquet('Data/Cleaned/countryindicators.parquet')

In [8]:
df_country_indicators.write.json('Data/Cleaned/countryindicators.jsonl')

In [48]:
df_country_indicators.count()

249

In [49]:
# Test if it is written properly
x = spark.read.parquet('Data/Cleaned/countryindicators.parquet')
print(x.count())
x.limit(3).toPandas()

249


,Country,ISOCode,TourismExpenditureMillions,TouristArrivalsThousands,Currency,ExchangeRateEndOfPeriod,GNIPerCapita,GDPPerCapita,MobilePhoneSubscriptions,NetMigrationRate,...,UrbanPopulationPercent,HDIRank,HDI,InternetUsersPercent,PoliticalRightsFreedomScore,CivilLibertiesFreedomScore,FreedomStatus,DemocracyOrNot,PoliticalRegimeTypeScore,HumanRightsScore
0,Afghanistan,AF,50.0,None,Afghani (AFN),74.9556,1746.0,1735.0,59.1,-1.7,...,25.5,170.0,0.496,None,5.0,6.0,Not Free,False,-1.0,-2.209409986
1,Åland Islands,AX,None,None,Euro (EUR),0.8734,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Albania,AL,2306.0,5340.0,Lek (ALL),107.82,12300.0,12306.0,94.2,-4.9,...,60.3,69.0,0.7909999999999999,None,3.0,3.0,Partly Free,True,9.0,0.7703089679999999


### Part 4: Write data sets to S3

In this section, we write the 4 data sets into a new S3 bucket. These data sets are:
* The Hotel reviews data set, df_hotel_reviews, prepared in this notebook.
* The Hotels data set, df_google_hotels_with_airportid, prepared in this notebook.
* The Airport codes data set, df_airport_codes, prepared in this notebook
* The CountryIndicators data set, prepared in the **CombineAdditionalCountryData** noteobok.

To write the data sets, I use the AWS CLI. The first step is to create a file ~/.aws/credentials with the following contents:
[default]
aws_access_key_id=XXX
aws_secret_access_key=XXX
aws_session_token=XXX

I use temporary credentials here.
I have given the CLI commands below. I use the sync command to upload the JSONL folders into S3 (Parquet doesn't seem to work well with Redshift).
#### Create BUCKET
aws s3 mb BUCKETNAME
#### Upload folders to S3
* aws s3 sync Data/Cleaned/hotels.parquet/ s3://BUCKETNAME/hotels.jsonl/
* aws s3 sync Data/Cleaned/countryindicators.parquet/ s3://BUCKETNAME/countryindicators.jsonl/
* aws s3 sync Data/Cleaned/airports.jsonl/ s3://BUCKETNAME/airports.josnl/
* aws s3 sync Data/Cleaned/reviews.jsonl/ s3://BUCKETNAME/reviews.jsonl/

#### Remove .crc files on s3 as they cannot be read by Redshift's copy command.
* aws s3 rm s3://hotelsreviewsandmore/hotels.jsonl --recursive --exclude "*" --include "*.crc"
* aws s3 rm s3://hotelsreviewsandmore/reviews.jsonl --recursive --exclude "*" --include "*.crc"
* aws s3 rm s3://hotelsreviewsandmore/countryindicators.jsonl --recursive --exclude "*" --include "*.crc"
* aws s3 rm s3://hotelsreviewsandmore/countryindicators.jsonl --recursive --exclude "*" --include "*.crc"

#### Create jsonpath files containing the schema and field order for all the 4 jsonl files
* See Data/Cleaned/JSONPath/ for the 4 jsonpath files.